# Overdraft model training notebook

## Importing relevant libraries 

In [ ]:
import sys
sys.path.append('../../.')

import pickle
import warnings
from datetime import datetime
import shap
import matplotlib.pyplot as plt
import numpy as np
from overdraft_prediction.model import ODModel
from overdraft_prediction.fine_tuner import FineTuner, ActionType, ThresholdType

import csv
import pandas as pd
import seaborn as sns
import snowflake.connector as sf
import xgboost as xgb
from dateutil.relativedelta import relativedelta
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split

## Loading environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")

## Setting pandas preferred display

In [ ]:
#set preferances

pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)
warnings.filterwarnings(action='once')
sns.set(rc={'axes.facecolor': 'white', 'figure.facecolor': 'white'})
sns.set_style("darkgrid")

## Connecting to SnowFlake

In [ ]:
conn = sf.connect(user=os.getenv('SNOWFLAKE_USER'),
                  password=os.getenv('SNOWFLAKE_PASSWORD'),
                  account=os.getenv('SNOWFLAKE_HOST')
                  )


def run_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close()


snowflakecursor = conn.cursor()
try:
    sql = 'alter warehouse {} resume'.format(os.getenv('SNOWFLAKE_WAREHOUSE'))
    run_query(conn, sql)
except:
    pass
sql = 'use database {}'.format(os.getenv('SNOWFLAKE_DATABASE'))
run_query(conn, sql)
sql = 'use database {}'.format(os.getenv('SNOWFLAKE_DATABASE'))

## Defining the functions to create training dataset

In [ ]:
def create_monthly_training_dateset(date_to_run):
    fd = date_to_run
        
    first_table_cols = '''
    select bank_account_id, count(distinct cl.id) as login_count
    from (select bank_account_id, customer_id from DWH.fact_mysql_customer_monthly group by 1,2) dw
    join "LILI_ANALYTICS_DEV"."ODS"."MYSQL_CUSTOMER_LOGIN" cl on cl.customer_id=dw.customer_id
    where create_time>=' ''' + fd + ''' ' and create_time<=dateadd(month, 1,' ''' + fd + ''' ')
    group by bank_account_id
    '''
    customer_login_per_date_range = pd.read_sql(first_table_cols, conn)

    second_table_cols = '''
    select BANK_ACCOUNT_ID, sum(TRANSACTION_AMOUNT) as total_money_in1
    from "LILI_ANALYTICS"."DWH"."FACT_MYSQL_ACCOUNT_TRANSACTION_ALL"
    where transaction_date>dateadd(month, 1, ' ''' + fd + ''' ') and transaction_date<=dateadd(month, 2,' ''' + fd + ''' ')
    and act_type in ('PM','AD') and transaction_amount>0
    group by BANK_ACCOUNT_ID
    '''
    
    bank_account_did_transactions = pd.read_sql(second_table_cols, conn)
    
    mcc_table_cols = '''
    select BANK_ACCOUNT_ID, MCC_CODE, sum(TRANSACTION_AMOUNT) as money_amount
    from "LILI_ANALYTICS"."DWH"."FACT_MYSQL_ACCOUNT_TRANSACTION_ALL"
    where transaction_date>=' ''' + fd + ''' ' and transaction_date<=dateadd(month, 1,' ''' + fd + ''' ')
    and TRANSACTION_AMOUNT<0
    group by BANK_ACCOUNT_ID, MCC_CODE
    '''
    mcc_table = pd.read_sql(mcc_table_cols, conn)
    mcc_table = pd.crosstab(mcc_table.BANK_ACCOUNT_ID, mcc_table.MCC_CODE, values=mcc_table.MONEY_AMOUNT, aggfunc='sum').fillna(0)

    pending_check_cols = '''
    SELECT bank_account_id,count(case when status=40 then id else null end ) as pending_check
    from  "LILI_ANALYTICS_DEV"."ODS"."MYSQL_ACCOUNT_PENDING_CHECK"
    where  BANK_ACCOUNT_ID is not null
    and CREATE_TIME>=' ''' + fd + ''' ' and CREATE_TIME<=dateadd(month, 1,' ''' + fd + ''' ')
    group by 1
    '''
    
    pending_check = pd.read_sql(pending_check_cols, conn)
    
    
    trans_rej_cols = '''
    select bank_account_id
    ,sum(case when SUB_TYPE='denied_auth_nsf' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth_nsf  
    ,count(case when SUB_TYPE='denied_auth_nsf' then bank_account_id else null end ) denied_auth_nsf_cnt 
    ,sum(case when SUB_TYPE='denied_auth' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth 
    ,count(case when SUB_TYPE='denied_auth' then bank_account_id else null end ) denied_auth_cnt 
    ,sum(case when SUB_TYPE='auth_exp' then abs(TRANSACTION_AMOUNT) else null end ) auth_exp  
    ,count(case when SUB_TYPE='auth_exp' then bank_account_id else null end ) auth_exp_cnt 
    ,sum(case when SUB_TYPE='denied_auth_inactive_card' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth_inactive_card  
    ,count(case when SUB_TYPE='denied_auth_inactive_card' then bank_account_id else null end ) denied_auth_inactive_card_cnt 
    ,sum(case when SUB_TYPE='denied_auth_invalid_pin' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth_invalid_pin 
    ,count(case when SUB_TYPE='denied_auth_invalid_pin' then bank_account_id else null end ) denied_auth_invalid_pin_cnt 
    ,sum(case when SUB_TYPE='denied_auth_gas' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth_gas
    ,count(case when SUB_TYPE='denied_auth_gas' then bank_account_id else null end ) denied_auth_gas_cnt 
    ,sum(case when SUB_TYPE='ach_credit_fail' then abs(TRANSACTION_AMOUNT) else null end ) ach_credit_fail
    ,count(case when SUB_TYPE='ach_credit_fail' then bank_account_id else null end ) ach_credit_fail_cnt
    ,sum(case when SUB_TYPE='ach_credit_return' then abs(TRANSACTION_AMOUNT) else null end ) ach_credit_return
    ,count(case when SUB_TYPE='ach_credit_return' then bank_account_id else null end ) ach_credit_return_cnt
    ,sum(case when SUB_TYPE='create_hold' then abs(TRANSACTION_AMOUNT) else null end ) create_hold
    ,count(case when SUB_TYPE='create_hold' then bank_account_id else null end ) create_hold_cnt
    ,sum(case when SUB_TYPE='expire_hold' then abs(TRANSACTION_AMOUNT) else null end ) expire_hold
    ,count(case when SUB_TYPE='expire_hold' then bank_account_id else null end ) expire_hold_cnt
    from "LILI_ANALYTICS"."ODS"."MYSQL_ACCOUNT_TRANSACTION_REJECT"
    WHERE TRANSACTION_DATE>=' ''' + fd + ''' ' and TRANSACTION_DATE<=dateadd(month, 1,' ''' + fd + ''' ')
    group by 1
    '''
    trans_rej = pd.read_sql(trans_rej_cols, conn)
    
    
    third_table_cols = '''
    select
    dw.customer_id,
    dw.bank_account_id,
    min(signup_date) as signup_date,
    dateadd(month, 1,' ''' + fd + ''' ') as period_end,
    max(transaction_date) as last_transaction_in_time,
    count(case when ata.rolling_balance < 0 then 1 else null end) as had_negative_balance,
    max(case when act_type='PM' then transaction_date else null end) as last_transaction_money_in_in_time,
    abs(sum(case when act_type in ('ST','VS','IS','VI', 'DB', 'SD', 'MP') and ata.type = 'W' then ata.transaction_amount else 0 end)) as ATM_sum,
    abs(sum(case when act_type in ('ST','VS','IS','VI', 'DB', 'SD', 'MP') and ata.type <> 'W' then ata.transaction_amount else 0 end)) as Swipe_sum,
    abs(sum(case when act_type in ('ST','VS','IS','VI', 'DB', 'SD', 'MP') then ata.transaction_amount else 0 end)) as Spend_sum,
    sum(case when act_type='PM' and ata.type='FM' then abs(ata.transaction_amount) else 0 end) - sum(case when (ata.type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(ata.transaction_amount) else 0 end) as direct_deposit_sum,
    sum(case when act_type='PM' and (ata.type in ('VT','VA','VH','MX')) then abs(ata.transaction_amount) else 0 end) as direct_pay_sum,
    abs(sum(case when (act_type='PM' and ata.type='AC') and ata.transaction_amount > 0 then ata.transaction_amount else 0 end)) as ACH_sum,
    sum(case when act_type='PM' and (ata.type='OR') then abs(ata.transaction_amount) else 0 end) as Check_sum,
    sum(case when act_type='PM' and (ata.type in ('GT', 'GO', 'CE')) then abs(ata.transaction_amount) else 0 end) as Greendot_sum,
    sum(case when (act_type='AD' and details='Debit Card transfer') then abs(ata.transaction_amount) else 0 end) as Card_Deposit_sum,
    sum(case when (act_type='PM' and ata.type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or ata.type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (ata.type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(ata.transaction_amount) else 0 end) as Total_money_in,
    sum(case when (act_type='PM' and ata.type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or ata.type='FM')) and transaction_amount > 1 and transaction_date >= dateadd(day, 14, ' ''' + fd + ''' ') then transaction_amount else null end) - sum(case when (ata.type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(ata.transaction_amount) else 0 end) as Total_money_in2,
    count(case when (act_type='PM' and ata.type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or ata.type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (ata.type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(ata.transaction_amount) else null end) as Total_money_in_Count,
    avg(ata.rolling_balance) as average_balance,
    count(distinct case when dds.type='PAYROLL' then 1 else null end) as did_payroll,
    count(distinct case when dds.type='Marketplace' then 1 else null end) as did_marketplace,
    count(distinct case when dds.type='FINANCIAL INSTITUTION' then 1 else null end) as did_financial_institution,
    count(distinct case when dds.type='Unemployment' then 1 else null end) as did_unemployment,
    count(distinct case when dds.type='Tax Refund' then 1 else null end) as did_tax_refund
    from (select customer_id, bank_account_id, signup_date, account_active from DWH.fact_mysql_customer_monthly group by 1,2,3,4) dw
    join DWH.fact_mysql_account_transaction_all ata on dw.bank_account_id=ata.bank_account_id and ata.transaction_date > ' ''' + fd + ''' ' and ata.transaction_date <= dateadd(month, 1, ' ''' + fd + ''' ') 
    left join ODS.mysql_direct_deposit_sources dds on dds.merchant=ata.details
    where account_active=1
    group by 1, 2
    having Total_money_in>500
    order by 1'''

    money_in_week1_cols = '''
    select
    bank_account_id,
    sum(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else 0 end) as Total_money_in_week1,
    count(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else null end) as Total_money_in_Count_week1
    from DWH.fact_mysql_account_transaction_all
    where transaction_date >= ' ''' + fd + ''' ' and transaction_date <= dateadd(day, -21, dateadd(month, 1, ' ''' + fd + ''' '))
    group by 1
    order by 1
    '''
    
    money_in_week1 = pd.read_sql(money_in_week1_cols, conn)    
    
    
    money_in_week2_cols = '''
    select
    bank_account_id,
    sum(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else 0 end) as Total_money_in_week2,
    count(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else null end) as Total_money_in_Count_week2
    from DWH.fact_mysql_account_transaction_all
    where transaction_date > dateadd(day, -21, dateadd(month, 1, ' ''' + fd + ''' ')) and transaction_date <= dateadd(day, -14, dateadd(month, 1, ' ''' + fd + ''' '))
    group by 1
    order by 1
    '''
    
    money_in_week2 = pd.read_sql(money_in_week2_cols, conn)
    
    money_in_week3_cols = '''
    select
    bank_account_id,
    sum(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else 0 end) as Total_money_in_week3,
    count(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else null end) as Total_money_in_Count_week3
    from DWH.fact_mysql_account_transaction_all
    where transaction_date > dateadd(day, -14, dateadd(month, 1, ' ''' + fd + ''' ')) and transaction_date <= dateadd(day, -7, dateadd(month, 1, ' ''' + fd + ''' '))
    group by 1
    order by 1
    '''
    
    money_in_week3 = pd.read_sql(money_in_week3_cols, conn)
    
    money_in_week4_cols = '''
    select
    bank_account_id,
    sum(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else 0 end) as Total_money_in_week4,
    count(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else null end) as Total_money_in_Count_week4
    from DWH.fact_mysql_account_transaction_all
    where transaction_date > dateadd(day, -7, dateadd(month, 1, ' ''' + fd + ''' ')) and transaction_date <= dateadd(month, 1, ' ''' + fd + ''' ') 
    group by 1
    order by 1
    ''' 
    
    money_in_week4 = pd.read_sql(money_in_week4_cols, conn)
    
    third_table = pd.read_sql(third_table_cols, conn)
    first_join = third_table.merge(customer_login_per_date_range, on=['BANK_ACCOUNT_ID'], how='left')
    second_join = first_join.merge(pending_check, on=['BANK_ACCOUNT_ID'], how='left')
    third_join = second_join.merge(trans_rej, on=['BANK_ACCOUNT_ID'], how='left')
    bank_account_did_transactions = bank_account_did_transactions.merge(mcc_table, on=['BANK_ACCOUNT_ID'], how='left')
    trans_table = third_join.merge(bank_account_did_transactions, on=['BANK_ACCOUNT_ID'], how='left')
    table_with_money_week1 = trans_table.merge(money_in_week1, on=['BANK_ACCOUNT_ID'], how='left')
    table_with_money_week2 = table_with_money_week1.merge(money_in_week2, on=['BANK_ACCOUNT_ID'], how='left')
    table_with_money_week3 = table_with_money_week2.merge(money_in_week3, on=['BANK_ACCOUNT_ID'], how='left')
    table_all = table_with_money_week3.merge(money_in_week4, on=['BANK_ACCOUNT_ID'], how='left')
    table_all.rename(columns={'TOTAL_MONEY_IN1': 'LABEL'}, inplace=True)
    table_all['LABEL'] = table_all['LABEL'].fillna(0)
    table_all['LOGIN_COUNT'] = table_all['LOGIN_COUNT'].fillna(0)
    table_all['TOTAL_MONEY_IN'] = table_all['TOTAL_MONEY_IN'].fillna(0)
    return table_all



def create_complete_training_set(start_date, end_date):
    date_dt_from = datetime.strptime(start_date, '%Y-%m-%d')
    date_dt_to = datetime.strptime(end_date, '%Y-%m-%d')
    future_date = date_dt_from
    table_all = []
    i = 0
    while future_date <= date_dt_to:
        print(f"Currently creating dataset of date: {str(future_date)}")
        current_date = str(future_date)
        current_date = current_date[0:-9]
        table_all.append(create_monthly_training_dateset(current_date))
        
        future_date = future_date + relativedelta(months=1)
        i = i + 1
    
    #regrouping all tables together and postprocessing
    sum = 0
    temp = table_all[0]
    for j in range(0, i):
        sum = sum + table_all[j].shape[0]
        if j >= 1:
            temp = pd.concat([temp, table_all[j]], ignore_index=True)
    print(f"total table columns: {str(sum)}")



    table_all = temp
    table_all['DID_NONE'] = (1 - table_all[
        ['DID_PAYROLL', 'DID_MARKETPLACE', 'DID_FINANCIAL_INSTITUTION', 'DID_UNEMPLOYMENT', 'DID_TAX_REFUND']].max(
        axis=1))
    table_all2 = table_all

    table_all['TIME_FROM_LAST_TRANSACTION'] = \
    (pd.to_datetime(table_all['PERIOD_END'], format="%Y%m%") - table_all['LAST_TRANSACTION_IN_TIME']) \
        .astype('timedelta64[D]') 
    table_all['TIME_FROM_LAST_MONEY_IN'] = \
    (pd.to_datetime(table_all['PERIOD_END'], format="%Y%m%") - table_all['LAST_TRANSACTION_MONEY_IN_IN_TIME']) \
        .astype('timedelta64[D]')
    table_all = table_all.drop(['SIGNUP_DATE', 'CUSTOMER_ID', 'BANK_ACCOUNT_ID', 'PERIOD_END', 'LAST_TRANSACTION_IN_TIME', \
                            'LAST_TRANSACTION_MONEY_IN_IN_TIME'], axis=1)
    table_all['AVG_MONEY_IN'] = np.where(table_all['TOTAL_MONEY_IN'] == 0.00, 0.00, 
                                     table_all['TOTAL_MONEY_IN'] / table_all['TOTAL_MONEY_IN_COUNT'])

    table_all.fillna(0)
    table_all= table_all.replace(np.nan,0)
    return table_all


def reorder_and_finalize_dataset(dataset):
    #make dataset columns in specific order and add mcc codes including missing ones
    #from MCC_LIST_FOR_OVERDRAFT table
    
    cols = list(table_all.columns)
    cols.sort()

    mcc_list_cols = '''
    select * from  "LILI_ANALYTICS_DEV"."ODS"."MCC_LIST_FOR_OVERDRAFT" order by MCC_CODE'''
    mcc_list = pd.read_sql(mcc_list_cols, conn)
    list1 = ['TOTAL_MONEY_IN', 'TOTAL_MONEY_IN_COUNT', 'TOTAL_MONEY_IN_WEEK1',\
    'TOTAL_MONEY_IN_COUNT_WEEK1', 'TOTAL_MONEY_IN_WEEK2',\
    'TOTAL_MONEY_IN_COUNT_WEEK2', 'TOTAL_MONEY_IN_WEEK3', \
    'TOTAL_MONEY_IN_COUNT_WEEK3', 'TOTAL_MONEY_IN_WEEK4',\
    'TOTAL_MONEY_IN_COUNT_WEEK4', 'HAD_NEGATIVE_BALANCE', 'ATM_SUM', 'SWIPE_SUM',\
    'SPEND_SUM', 'DIRECT_DEPOSIT_SUM', 'DIRECT_PAY_SUM', 'ACH_SUM', 'CHECK_SUM', 'GREENDOT_SUM',\
    'CARD_DEPOSIT_SUM', 'AVERAGE_BALANCE', 'DID_PAYROLL', 'DID_MARKETPLACE', 'DID_FINANCIAL_INSTITUTION',\
    'DID_UNEMPLOYMENT', 'DID_TAX_REFUND', 'DID_NONE', 'LOGIN_COUNT', 'PENDING_CHECK', 'DENIED_AUTH_NSF',\
    'DENIED_AUTH_NSF_CNT', 'DENIED_AUTH', 'DENIED_AUTH_CNT', 'AUTH_EXP', 'AUTH_EXP_CNT', \
    'DENIED_AUTH_INACTIVE_CARD', 'DENIED_AUTH_INACTIVE_CARD_CNT', 'DENIED_AUTH_INVALID_PIN',\
    'DENIED_AUTH_INVALID_PIN_CNT', 'DENIED_AUTH_GAS', 'DENIED_AUTH_GAS_CNT', 'ACH_CREDIT_FAIL',\
    'ACH_CREDIT_FAIL_CNT', 'ACH_CREDIT_RETURN', 'ACH_CREDIT_RETURN_CNT', 'CREATE_HOLD', 'CREATE_HOLD_CNT',\
    'EXPIRE_HOLD', 'EXPIRE_HOLD_CNT', 'TIME_FROM_LAST_TRANSACTION', 'TIME_FROM_LAST_MONEY_IN', 'AVG_MONEY_IN',\
    'LABEL']

    num_all = -len(list1)-1
    current_mccs = cols[:num_all]

    missing_mcc = list(set(mcc_list['MCC_CODE']) - set(current_mccs))

    for elt in missing_mcc:
        table_all[elt] = 0
    list2 = list(mcc_list['MCC_CODE'].iloc[0:])
    listush = list1 + list2
    table_ready = table_all[listush]
    return table_ready



## Creating training dataset

In [ ]:
date_from = '2020-04-01'
date_to = '2021-07-01' #taking 3 at most months back from current month
table_all = create_complete_training_set(date_from, date_to)
table_all

## Some additional necessary functions for later on

In [ ]:
def money_loss(labels, predictions, balance_up_amounts):
    money = 0
    for i in range(0, len(labels)):
        if predictions[i]==4 and labels[i]==0:
            money = money + (balance_up_amounts[4] - balance_up_amounts[0])
        if predictions[i]==4 and labels[i]==1:
            money = money + (balance_up_amounts[4] - balance_up_amounts[1])
        if predictions[i]==4 and labels[i]==2:
            money = money + (balance_up_amounts[4] - balance_up_amounts[2])
        if predictions[i]==4 and labels[i]==3:
            money = money + (balance_up_amounts[4] - balance_up_amounts[3])
        if predictions[i]==3 and labels[i]==0:
            money = money + (balance_up_amounts[3] - balance_up_amounts[0])
        if predictions[i]==3 and labels[i]==1:
            money = money + (balance_up_amounts[3] - balance_up_amounts[1])
        if predictions[i]==3 and labels[i]==2:
            money = money + (balance_up_amounts[3] - balance_up_amounts[2])
        if predictions[i]==2 and labels[i]==0:
            money = money + (balance_up_amounts[2] - balance_up_amounts[0])
        if predictions[i]==2 and labels[i]==1:
            money = money + (balance_up_amounts[2] - balance_up_amounts[1])
        if predictions[i]==1 and labels[i]==0:
            money = money + (balance_up_amounts[1] - balance_up_amounts[0])
    return money

def adjust_output_to_labels(label_feature, overdraft_percent_dict):
    #currently the specs are as follows 20 - 50%, 40 - 30%, 60 - 15%, 100 - 5%
    ranges = []
    percents = []
    od_values = list(overdraft2percent.keys())
    for elt in od_values:
        percents.append(overdraft2percent[elt])
    #add the first element from the keys so being under this threshold will be considered 0 overdraft
    ranges.append(od_values[0])
    specs = label_feature.describe(np.linspace(0, 1, 101))
    for i in range(0,len(specs)):
        for j in range(1,len(percents)):
            if specs.keys()[i][0:-1]==str(np.sum(percents[0:j])):
                ranges.append(int(specs.values[i]))
    return ranges

def convert_label_to_numerical_ranges(dataset, label_ranges):
    labeler = dataset['LABEL']
    for i in range(0, len(labeler)):
        
        if labeler[i] < label_ranges[0]:
            labeler[i]=0
            continue
        for j in range(1, len(label_ranges)):
            if labeler[i] >= label_ranges[j-1] and labeler[i]<label_ranges[j]:
                labeler[i]=j
                continue
        if labeler[i]>=label_ranges[-1]:
            labeler[i]=len(label_ranges)
    return labeler

## Finalizing dataset and defining label ranges

In [ ]:
#dictionary between overdraft and precentage from users money-in for prediction
overdraft2percent = {20: 50, 40: 30, 60: 15, 100: 5}

table_general = reorder_and_finalize_dataset(table_all)
table_general

label_outputs = adjust_output_to_labels(table_general['LABEL'], overdraft2percent)
print(f"ranges for balance-up values in USD are: {label_outputs}")
labelers = convert_label_to_numerical_ranges(table_general, label_outputs)
table_general['LABEL'] = labelers

## Sanity check for ranges

In [ ]:
cumlate = []
for i in range(0, len(overdraft2percent)+1):
    cumlate.append(table_general[table_general['LABEL'] == i].shape[0])
    print(f"For label {i} we have {cumlate[-1]} date points")
    
print(f"Total labeled ranges are {np.sum(cumlate)}") 

if np.sum(cumlate)!=table_general.shape[0]:
    print("Error! Something is wrong. Total number of date point does not equal to labeled ranges")

## Extract features for training with train-test division

In [ ]:
#Let's modelize!

#First step - make seperate labels from features and convert to numpy arrays

# Labels are the values we want to predict
labels = np.array(table_general['LABEL'])
# Remove the labels from the features
# axis 1 refers to the columns
features = table_general.drop('LABEL', axis=1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)
features.shape
# Split to training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25)

## Train model using XGBoost

In [ ]:
#Third step - train model

model_overdraft = xgb.XGBClassifier(use_label_encoder=False, max_depth=3, n_estimators=250, learning_rate=0.3)
model_overdraft.fit(train_features, train_labels)

## Check model accuracy and relative accuracy (equal or lower prediction) and present the confusion matrix

### In particular, the relative accuracy is in the lower triagular values of the confusion matrix

In [ ]:
predictions = model_overdraft.predict(test_features)
print("Accuracy:", metrics.accuracy_score(predictions, test_labels))

comparush = (predictions <= test_labels)
unique, counts = np.unique(comparush, return_counts=True)
print("The predictions portions that were equal or less: " + str(counts[1] / (counts[0] + counts[1])))
#the current algorithm gives 85.5% of clients overdraft less or equal to what they should
disp = plot_confusion_matrix(model_overdraft, test_features, test_labels, display_labels=['0', '20', '40', '60', '100'],
                             cmap=plt.cm.Blues, values_format='.1f')

## Present feature importance for this model training (from highest to lowest)

In [ ]:
# Get numerical feature importances
importances = list(model_overdraft.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation='vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance');
plt.xlabel('Variable');
plt.title('Variable Importances');

## Adding shap explainer object for model reason (not obligatory)

In [ ]:
explainer = shap.TreeExplainer(model_overdraft, train_features, feature_perturbation="interventional")


## Adding finetune object for model reason (not obligatory)

### Alittle bit more information regarding finetune. This object helps the model be a little bit more careful on some situations and alittle bit more hesitant on others. The way the finetune model works is, it takes the prediction probabilities and sets thresholds on these probabilities, such that when a prediction probability threshold is set, the output will change accordingly. For additional information, check the test_fine_tuner.py file, there are good examples there. To find good thresholds, it can be a good practice to run the money_loss function defined above

In [ ]:
example_fine_tuner1 = FineTuner(0, 0.3, ThresholdType.MORE_THAN, ActionType.NEXT_BEST)
example_fine_tuner2 = FineTuner(1, 0.55, ThresholdType.LESS_THAN, ActionType.ABSOLUTE, 3)

current_overdraft_limits = [0, 20, 40, 60, 100]
loss_usd = money_loss(test_labels, predictions, current_overdraft_limits)
print(f"money currently lost is {loss_usd}")


## NOTE!!! Model rejection, although exists as object is currently not used. I believe that in the future it will be used, this is why in the ODModel object it can be used in input. For reference and some code reusability, I kept the old model reject code down below

## Saving the model to pkl file. ODModel inputs are: model_money_in, model_reject (Not obligatory), finetuner (Not obligatory), explainer  (Not obligatory)

In [ ]:
import datetime
#creating and saving the ODmodel
#currently we just put balance-up model and explainer object as inputs
odmodel = ODModel(model_money_in=model_overdraft, explainer=explainer)
filename = f'../../model_dumps/ODmodel_{datetime.datetime.utcnow().isoformat()}.pkl'
pickle.dump(odmodel, open(filename, 'wb'))
print(f"model is saved in {filename}")

In [ ]:
if 'explainerc' in vars() or 'explainerc' in globals():
    print("exists")
else:
    print("does not exist")

In [ ]:
odmodel.__dict__

In [ ]:
#Load a model if necessary
modeler = pickle.load(open('../../model_dumps/ODmodel_2021-08-17T15:05:04.740356.pkl', 'rb'))
explainush = modeler.explainer
print(dir(modeler))
modeler.__dict__

In [ ]:
#confusion matrix
print(metrics.classification_report(test_labels, predictions))
disp = plot_confusion_matrix(model_overdraft, test_features, test_labels, display_labels=['0', '20', '40', '60', '100'],
                             cmap=plt.cm.Blues, values_format='.1f')
plt.show()

In [ ]:
count = 0
for i in range(0, len(predictions)):
    if test_labels[i]==2 and predictions[i]==3:
        count = count + 1
print(count)
    

In [ ]:
#predictions2.shape
preds_proba = model_overdraft.predict_proba(test_features)

In [ ]:
predictions[preds_proba[:, 0] > 0.2]

In [ ]:
from sklearn.metrics import confusion_matrix

print(labels)
sns.heatmap(confusion_matrix(test_labels, predictions), cmap="Blues", annot=True, fmt=".2f")


In [ ]:
preds_proba[preds_proba[:, 0] > 0.2]

In [ ]:
print(classification_report(test_labels, predictions, target_names=['200', '100', '50', '20', 'NO']))

In [ ]:
print(classification_report(test_labels, predictions, target_names=['200', '100', '50', '20', 'NO']))

In [ ]:
#Let's take the five highest and try to run the algorithm again
table2 = table_ready[['LABEL', 'TIME_FROM_LAST_TRANSACTION',
                      'TOTAL_MONEY_IN',
                      'TOTAL_MONEY_IN_COUNT',
                      'DID_UNEMPLOYMENT',
                      'DIRECT_DEPOSIT_SUM',
                      'DID_TAX_REFUND', 'DID_PAYROLL', 'DID_NONE']]

In [ ]:
#Let's modelize!

#First step - make seperate labels from features and convert to numpy arrays

# Labels are the values we want to predict
labels2 = np.array(table2['LABEL'])
# Remove the labels from the features
# axis 1 refers to the columns
features2 = table2.drop('LABEL', axis=1)
# Saving feature names for later use
feature_list2 = list(features2.columns)
# Convert to numpy array
features2 = np.array(features2)

In [ ]:
# Second step - split the data into training and testing sets

train_features, test_features, train_labels, test_labels = train_test_split(features2, labels2, test_size=0.25)

In [ ]:
#Third step - train model


model = xgb.XGBClassifier(max_depth=3, n_estimators=250, learning_rate=0.24)
model.fit(train_features, train_labels)

In [ ]:
predictions = model.predict(test_features)
print("Accuracy:", metrics.accuracy_score(predictions, test_labels))
comparush = (predictions >= test_labels)
unique, counts = np.unique(comparush, return_counts=True)
print("the predictions that were at least equal or worse are " + str(counts[1] / (counts[0] + counts[1])))

In [ ]:
disp = plot_confusion_matrix(model_overdraft, test_features, test_labels, display_labels=['NO', '20', '40', '60', '100'],
                             cmap=plt.cm.Blues, values_format='.1f')
plt.show()

In [ ]:
count=0

for i in range(0,len(test_features)):
    if predictions[i]==1 and test_labels[i]==2:
        count=count+1
        
print(count)

In [ ]:
print(classification_report(test_labels, predictions, target_names=['NO', '20', '40', '60', '100']))

In [ ]:
table_ready['LABEL'].hist(bins=50)

table_ready['LABEL'].describe(np.linspace(0, 1, 51))

In [ ]:
preds_proba = model_overdraft.predict_proba(test_features)
ranger = np.arange(start=0.20, stop=0.55, step=0.05)

In [ ]:
lst = []
for i in range(0, len(test_features)):
    chosen_cat = np.argmax(preds_proba[i])
    if chosen_cat==3:
        lst.append(max(preds_proba[i]))

lst = np.array(lst)
print(np.min(lst))
print(np.median(lst))
print(np.mean(lst))
print(np.max(lst))

In [ ]:
predictions2 = model_overdraft.predict(test_features)
for i in range(0, len(test_features)):
    chosen_cat = np.argmax(preds_proba[i])
    if chosen_cat==4 and max(preds_proba[i])>0.525:
        predictions2[i] = np.argmax([preds_proba[i][0],preds_proba[i][1],preds_proba[i][2],preds_proba[i][3]])
        

In [ ]:
plt.plot(ranger, full[0])
plt.xlabel("Prediction Probability Threshold")
plt.ylabel("Probability for true prediction above threshold")

In [ ]:
plt.plot(ranger, full[1])
plt.xlabel("Prediction Probability Threshold")
plt.ylabel("Probability for true prediction above threshold")

In [ ]:
plt.plot(ranger, full[2])
plt.xlabel("Prediction Probability Threshold")
plt.ylabel("Probability for true prediction above threshold")

In [ ]:
plt.plot(ranger, full[3])
plt.xlabel("Prediction Probability Threshold")
plt.ylabel("Probability for true prediction above threshold")

In [ ]:
plt.plot(ranger, full[4])
plt.xlabel("Prediction Probability Threshold")
plt.ylabel("Probability for true prediction above threshold")

In [ ]:
#fine-tuning for the model
predictions2 = model.predict(test_features)
comparush = (predictions >= test_labels)
unique, counts = np.unique(comparush, return_counts=True)
orig_eob = counts[1] / (counts[0] + counts[1])
orig_acc = metrics.accuracy_score(predictions, test_labels)
print(orig_acc)
print(orig_eob)
rng = np.arange(start=0.3, stop=0.65, step=0.02)
twks = []
for j in range(0, len(rng)):
    for k in range(0, len(rng)):
        for l in range(0, len(rng)):
            for i in range(0, len(preds_proba)):
                chosen_cat = np.argmax(preds_proba[i]) - 5
                prob = max(preds_proba[i])
                if prob <= rng[j] and chosen_cat == -5:
                    predictions2[i] = np.argmax(
                        [preds_proba[i][1], preds_proba[i][2], preds_proba[i][3], preds_proba[i][4]]) - 4
                if prob <= rng[k] and chosen_cat == -4:
                    predictions2[i] = np.argmax([preds_proba[i][2], preds_proba[i][3], preds_proba[i][4]]) - 3
                if prob <= rng[l] and chosen_cat == -3:
                    predictions2[i] = np.argmax([preds_proba[i][3], preds_proba[i][4]]) - 2
            new_acc = metrics.accuracy_score(predictions2, test_labels)
            comparush2 = (predictions2 >= test_labels)
            unique, counts = np.unique(comparush2, return_counts=True)
            new_eob = counts[1] / (counts[0] + counts[1])
            #print("for threshold of [{:01f},{:01f},{:01f}] we get acc diff of {:02f} and eob diff of {:02f}.".format(rng[j],rng[k],rng[l], (new_acc-orig_acc)*100,\
            #                                (new_eob-orig_eob)*100))
            #print("ratio is {:02f}".format((new_eob-orig_eob)/(new_acc-orig_acc)))
            #print(new_acc)
            #print(new_eob)
            twks.append([rng[j], rng[k], rng[l], new_acc, new_eob])


In [ ]:
for i in range(0, len(preds_proba)):
    chosen_cat = np.argmax(preds_proba[i]) - 5
    prob = max(preds_proba[i])
    if prob <= 0.432 and chosen_cat == -5:
        predictions2[i] = np.argmax([preds_proba[i][1], preds_proba[i][2], preds_proba[i][3], preds_proba[i][4]]) - 4
    if prob <= 0.49 and chosen_cat == -4:
        predictions2[i] = np.argmax([preds_proba[i][2], preds_proba[i][3], preds_proba[i][4]]) - 3
    if prob <= 0.36 and chosen_cat == -3:
        predictions2[i] = np.argmax([preds_proba[i][3], preds_proba[i][4]]) - 2


In [ ]:
predictions = model.predict(test_features)
print("Original Accuracy:", metrics.accuracy_score(predictions, test_labels))
comparush = (predictions >= test_labels)
unique, counts = np.unique(comparush, return_counts=True)
print("the predictions that were at least equal or worse are " + str(counts[1] / (counts[0] + counts[1])))

print("Altered Accuracy:", metrics.accuracy_score(predictions2, test_labels))
comparush = (predictions2 >= test_labels)
unique, counts = np.unique(comparush, return_counts=True)
print("the predictions that were at least equal or worse are " + str(counts[1] / (counts[0] + counts[1])))


In [ ]:
from sklearn.metrics import confusion_matrix

print(labels)
sns.heatmap(confusion_matrix(test_labels, predictions), cmap="Blues", annot=True, fmt=".2f"
           , xticklabels=[0,20,40,60,100], yticklabels= [0,20,40,60,100])

np.histogram(preds_proba[:, 3])

In [ ]:
from sklearn.metrics import confusion_matrix

print(labels)
sns.heatmap(confusion_matrix(test_labels, predictions2), cmap="Blues", annot=True, fmt=".2f"
            , xticklabels=[0,20,40,60,100], yticklabels= [0,20,40,60,100])
np.histogram(preds_proba[:, 3])

In [ ]:
vec = np.arange(0.5,1,0.01)
vec2 = []
for elt in vec:
    predictions2 = model_overdraft.predict(test_features)
    for i in range(0, len(test_features)):
        chosen_cat = np.argmax(preds_proba[i])
        if chosen_cat==3 and max(preds_proba[i])<elt:
            predictions2[i] = np.argmax([preds_proba[i][0],preds_proba[i][1],preds_proba[i][2]])
    vec2.append(count_loss(test_labels, predictions2))
#print(count_loss(test_labels, predictions))
#print(count_loss(test_labels, predictions2))


In [ ]:
plt.plot(vec,vec2)

In [ ]:
predictions2 = model_overdraft.predict(test_features)
for i in range(0, len(test_features)):
    chosen_cat = np.argmax(preds_proba[i])
    if chosen_cat==4 and max(preds_proba[i])<0.68:
        predictions2[i] = np.argmax([preds_proba[i][0],preds_proba[i][1],preds_proba[i][2],preds_proba[i][3]])
    if chosen_cat==3 and max(preds_proba[i])<0.58:
        predictions2[i] = np.argmax([preds_proba[i][0],preds_proba[i][1],preds_proba[i][2]])

print(count_loss(test_labels, predictions))
print(count_loss(test_labels, predictions2))

In [ ]:
table_all2['LABEL'].describe(np.linspace(0, 1, 101))

# IN THIS PART WE TRAIN THE REJECTION MODEL, WHICH TAKES INPUT AND SAYS IF BEHAVIOR OF CUSTOMER IS STRANGE

In [ ]:
OD_cols = ''' select *  from "LILI_ANALYTICS_DEV"."ODS"."OVERDRAFT_LIMITS" '''

OD_table = pd.read_sql(OD_cols, conn)
#OD_table[OD_table['OVERDRAFT_LIMIT']=='40.0000']
OD_table

## Take all relevant dates

In [ ]:
dates_cols = '''
select DISTINCT EXPIRATION_DATE from "LILI_ANALYTICS_DEV"."ODS"."LAST_7_DAYS_OVERDRAFT_USAGE" 
'''
orig_dates=[]
run_dates=[]
dates = pd.read_sql(dates_cols, conn)
dates = list(dates['EXPIRATION_DATE'])
for i in range(0, len(dates)):
    run_dates.append(dates[i] + relativedelta(months=-2))
    orig_dates.append(dates[i])

    
for i in range(0, len(orig_dates)):
    new_query_cols = '''
    select * from "LILI_ANALYTICS_DEV"."ODS"."LAST_7_DAYS_OVERDRAFT_USAGE"
    WHERE EXPIRATION_DATE = ''' + str("'" + str(orig_dates[i]) + "'")

    new_query = pd.read_sql(new_query_cols, conn)
    print(new_query.shape[0])
    
rel_prns_cols = '''
select BANK_ACCOUNT_NUMBER from "LILI_ANALYTICS_DEV"."ODS"."LAST_7_DAYS_OVERDRAFT_USAGE" 
'''

rel_prns = pd.read_sql(rel_prns_cols, conn)
rel_prns = list(rel_prns['BANK_ACCOUNT_NUMBER'])
rel_prns

In [ ]:
table_rej_all = []
for i in range(0, len(run_dates)+1):
    if i<len(run_dates):
        fd = str(run_dates[i])
    else:
        fd = '2021-07-01'
    first_table_cols = '''
        select bank_account_id, count(distinct cl.id) as login_count, max(pro_customer) as pro_customer
        from (select bank_account_id, customer_id, max(pro_customer) as pro_customer from DWH.fact_mysql_customer_monthly group by 1,2) dw
        join "LILI_ANALYTICS_DEV"."ODS"."MYSQL_CUSTOMER_LOGIN" cl on cl.customer_id=dw.customer_id
        where create_time>=' ''' + fd + ''' ' and create_time<=dateadd(month, 1,' ''' + fd + ''' ')
        group by bank_account_id
    '''
    customer_login_per_date_range = pd.read_sql(first_table_cols, conn)

    
    mcc_table_cols = '''
        select BANK_ACCOUNT_ID, MCC_CODE, sum(TRANSACTION_AMOUNT) as money_amount
        from "LILI_ANALYTICS"."DWH"."FACT_MYSQL_ACCOUNT_TRANSACTION_ALL"
        where transaction_date>=' ''' + fd + ''' ' and transaction_date<=dateadd(month, 1,' ''' + fd + ''' ')
        and TRANSACTION_AMOUNT<0
        group by BANK_ACCOUNT_ID, MCC_CODE
    '''
    mcc_table = pd.read_sql(mcc_table_cols, conn)
    mcc_table = pd.crosstab(mcc_table.BANK_ACCOUNT_ID, mcc_table.MCC_CODE, values=mcc_table.MONEY_AMOUNT, aggfunc='sum').fillna(0)

    pending_check_cols = '''
    SELECT bank_account_id,count(case when status=40 then id else null end ) as pending_check
    from  "LILI_ANALYTICS_DEV"."ODS"."MYSQL_ACCOUNT_PENDING_CHECK"
    where  BANK_ACCOUNT_ID is not null
    and CREATE_TIME>=' ''' + fd + ''' ' and CREATE_TIME<=dateadd(month, 1,' ''' + fd + ''' ')
    group by 1
    '''
    
    pending_check = pd.read_sql(pending_check_cols, conn)
    
    
    trans_rej_cols = '''
    select bank_account_id
    ,sum(case when SUB_TYPE='denied_auth_nsf' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth_nsf  
    ,count(case when SUB_TYPE='denied_auth_nsf' then bank_account_id else null end ) denied_auth_nsf_cnt 
    ,sum(case when SUB_TYPE='denied_auth' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth 
    ,count(case when SUB_TYPE='denied_auth' then bank_account_id else null end ) denied_auth_cnt 
    ,sum(case when SUB_TYPE='auth_exp' then abs(TRANSACTION_AMOUNT) else null end ) auth_exp  
    ,count(case when SUB_TYPE='auth_exp' then bank_account_id else null end ) auth_exp_cnt 
    ,sum(case when SUB_TYPE='denied_auth_inactive_card' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth_inactive_card  
    ,count(case when SUB_TYPE='denied_auth_inactive_card' then bank_account_id else null end ) denied_auth_inactive_card_cnt 
    ,sum(case when SUB_TYPE='denied_auth_invalid_pin' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth_invalid_pin 
    ,count(case when SUB_TYPE='denied_auth_invalid_pin' then bank_account_id else null end ) denied_auth_invalid_pin_cnt 
    ,sum(case when SUB_TYPE='denied_auth_gas' then abs(TRANSACTION_AMOUNT) else null end ) denied_auth_gas
    ,count(case when SUB_TYPE='denied_auth_gas' then bank_account_id else null end ) denied_auth_gas_cnt 
    ,sum(case when SUB_TYPE='ach_credit_fail' then abs(TRANSACTION_AMOUNT) else null end ) ach_credit_fail
    ,count(case when SUB_TYPE='ach_credit_fail' then bank_account_id else null end ) ach_credit_fail_cnt
    ,sum(case when SUB_TYPE='ach_credit_return' then abs(TRANSACTION_AMOUNT) else null end ) ach_credit_return
    ,count(case when SUB_TYPE='ach_credit_return' then bank_account_id else null end ) ach_credit_return_cnt
    ,sum(case when SUB_TYPE='create_hold' then abs(TRANSACTION_AMOUNT) else null end ) create_hold
    ,count(case when SUB_TYPE='create_hold' then bank_account_id else null end ) create_hold_cnt
    ,sum(case when SUB_TYPE='expire_hold' then abs(TRANSACTION_AMOUNT) else null end ) expire_hold
    ,count(case when SUB_TYPE='expire_hold' then bank_account_id else null end ) expire_hold_cnt
    from "LILI_ANALYTICS"."ODS"."MYSQL_ACCOUNT_TRANSACTION_REJECT"
    WHERE TRANSACTION_DATE>=' ''' + fd + ''' ' and TRANSACTION_DATE<=dateadd(month, 1,' ''' + fd + ''' ')
    group by 1
    '''
    trans_rej = pd.read_sql(trans_rej_cols, conn)
    
    
    third_table_cols = '''
    select
    dw.customer_id,
    dw.bank_account_id,
    min(signup_date) as signup_date,
    dateadd(month, 1,' ''' + fd + ''' ') as period_end,
    max(transaction_date) as last_transaction_in_time,
    count(case when ata.rolling_balance < 0 then 1 else null end) as had_negative_balance,
    max(case when act_type='PM' then transaction_date else null end) as last_transaction_money_in_in_time,
    abs(sum(case when act_type in ('ST','VS','IS','VI', 'DB', 'SD', 'MP') and ata.type = 'W' then ata.transaction_amount else 0 end)) as ATM_sum,
    abs(sum(case when act_type in ('ST','VS','IS','VI', 'DB', 'SD', 'MP') and ata.type <> 'W' then ata.transaction_amount else 0 end)) as Swipe_sum,
    abs(sum(case when act_type in ('ST','VS','IS','VI', 'DB', 'SD', 'MP') then ata.transaction_amount else 0 end)) as Spend_sum,
    sum(case when act_type='PM' and ata.type='FM' then abs(ata.transaction_amount) else 0 end) - sum(case when (ata.type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(ata.transaction_amount) else 0 end) as direct_deposit_sum,
    sum(case when act_type='PM' and (ata.type in ('VT','VA','VH','MX')) then abs(ata.transaction_amount) else 0 end) as direct_pay_sum,
    abs(sum(case when (act_type='PM' and ata.type='AC') and ata.transaction_amount > 0 then ata.transaction_amount else 0 end)) as ACH_sum,
    sum(case when act_type='PM' and (ata.type='OR') then abs(ata.transaction_amount) else 0 end) as Check_sum,
    sum(case when act_type='PM' and (ata.type in ('GT', 'GO', 'CE')) then abs(ata.transaction_amount) else 0 end) as Greendot_sum,
    sum(case when (act_type='AD' and details='Debit Card transfer') then abs(ata.transaction_amount) else 0 end) as Card_Deposit_sum,
    sum(case when (act_type='PM' and ata.type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or ata.type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (ata.type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(ata.transaction_amount) else 0 end) as Total_money_in,
    count(case when (act_type='PM' and ata.type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or ata.type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (ata.type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(ata.transaction_amount) else null end) as Total_money_in_Count,
    avg(ata.rolling_balance) as average_balance,
    count(distinct case when dds.type='PAYROLL' then 1 else null end) as did_payroll,
    count(distinct case when dds.type='Marketplace' then 1 else null end) as did_marketplace,
    count(distinct case when dds.type='FINANCIAL INSTITUTION' then 1 else null end) as did_financial_institution,
    count(distinct case when dds.type='Unemployment' then 1 else null end) as did_unemployment,
    count(distinct case when dds.type='Tax Refund' then 1 else null end) as did_tax_refund
    from (select customer_id, bank_account_id, signup_date, account_active from DWH.fact_mysql_customer_monthly group by 1,2,3,4) dw
    join DWH.fact_mysql_account_transaction_all ata on dw.bank_account_id=ata.bank_account_id and ata.transaction_date > ' ''' + fd + ''' ' and ata.transaction_date <= dateadd(month, 1, ' ''' + fd + ''' ') 
    left join ODS.mysql_direct_deposit_sources dds on dds.merchant=ata.details
    where account_active=1
    group by 1, 2
    order by 1'''


    added_bank_account_num_cols = '''
    SELECT BANK_ACCOUNT_ID, 
            MAX(BANK_ACCOUNT_NUMBER) AS BANK_ACCOUNT_NUMBER, 
            MAX((CASE WHEN PRODUCT_ID = 20643 THEN 1 ELSE 0 END)) AS IS_PRO_SINCE_SIGNUP 
    FROM "LILI_ANALYTICS"."ODS"."MYSQL_DW_CUSTOMER_MONTHLY_NEW"
    GROUP BY BANK_ACCOUNT_ID
    '''
    prn_table = pd.read_sql(added_bank_account_num_cols, conn)


    money_in_week1_cols = '''
    select
    bank_account_id,
    sum(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else 0 end) as Total_money_in_week1,
    count(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else null end) as Total_money_in_Count_week1
    from DWH.fact_mysql_account_transaction_all
    where transaction_date >= ' ''' + fd + ''' ' and transaction_date <= dateadd(day, -21, dateadd(month, 1, ' ''' + fd + ''' '))
    group by 1
    order by 1
    '''
    
    money_in_week1 = pd.read_sql(money_in_week1_cols, conn)    
    
    
    money_in_week2_cols = '''
    select
    bank_account_id,
    sum(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else 0 end) as Total_money_in_week2,
    count(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else null end) as Total_money_in_Count_week2
    from DWH.fact_mysql_account_transaction_all
    where transaction_date > dateadd(day, -21, dateadd(month, 1, ' ''' + fd + ''' ')) and transaction_date <= dateadd(day, -14, dateadd(month, 1, ' ''' + fd + ''' '))
    group by 1
    order by 1
    '''
    
    money_in_week2 = pd.read_sql(money_in_week2_cols, conn)
    
    money_in_week3_cols = '''
    select
    bank_account_id,
    sum(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else 0 end) as Total_money_in_week3,
    count(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else null end) as Total_money_in_Count_week3
    from DWH.fact_mysql_account_transaction_all
    where transaction_date > dateadd(day, -14, dateadd(month, 1, ' ''' + fd + ''' ')) and transaction_date <= dateadd(day, -7, dateadd(month, 1, ' ''' + fd + ''' '))
    group by 1
    order by 1
    '''
    
    money_in_week3 = pd.read_sql(money_in_week3_cols, conn)
    
    money_in_week4_cols = '''
    select
    bank_account_id,
    sum(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount else null end) - sum(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else 0 end) as Total_money_in_week4,
    count(case when (act_type='PM' and type<>'C2') or (act_type='AD' and (details='Debit Card transfer' or type='FM')) and transaction_amount > 1 then transaction_amount  else null end) - count(case when (type='FM' and act_type='AD' and details='Direct Deposit Return') then abs(transaction_amount) else null end) as Total_money_in_Count_week4
    from DWH.fact_mysql_account_transaction_all
    where transaction_date > dateadd(day, -7, dateadd(month, 1, ' ''' + fd + ''' ')) and transaction_date <= dateadd(month, 1, ' ''' + fd + ''' ') 
    group by 1
    order by 1
    ''' 
    
    money_in_week4 = pd.read_sql(money_in_week4_cols, conn)

    last_balance_cols = '''
    SELECT t.BANK_ACCOUNT_ID, t.CURRENT_BALANCE
        FROM "LILI_ANALYTICS_DEV"."ODS"."MYSQL_CUSTOMER_BALANCE_HISTORY" t
        INNER JOIN (
        SELECT BANK_ACCOUNT_ID, MAX(VALID_DATE) AS MAXDATE
        FROM "LILI_ANALYTICS_DEV"."ODS"."MYSQL_CUSTOMER_BALANCE_HISTORY" tm
        GROUP BY BANK_ACCOUNT_ID
        ) tm ON t.BANK_ACCOUNT_ID = tm.BANK_ACCOUNT_ID AND t.VALID_DATE = tm.MAXDATE
    '''

    last_balance = pd.read_sql(last_balance_cols, conn)
    if i!=len(orig_dates):
        labling_cols = '''
        select * from "LILI_ANALYTICS_DEV"."ODS"."LAST_7_DAYS_OVERDRAFT_USAGE"
        WHERE EXPIRATION_DATE = ''' + str("'" + str(orig_dates[i]) + "'")
    
        labling = pd.read_sql(labling_cols, conn)
        labling['LABEL']=1
        labling = labling[['LABEL', 'BANK_ACCOUNT_NUMBER','EXPIRATION_DATE']]
    third_table = pd.read_sql(third_table_cols, conn)
    print(third_table.shape)
    first_join  = third_table.merge(customer_login_per_date_range, on=['BANK_ACCOUNT_ID'], how='left')
    print(first_join.shape)
    second_join = first_join.merge(pending_check, on=['BANK_ACCOUNT_ID'], how='left')
    print(second_join.shape)
    third_join  = second_join.merge(prn_table, on=['BANK_ACCOUNT_ID'], how='left')
    print(third_join.shape)
    fourth_join = third_join.merge(trans_rej, on=['BANK_ACCOUNT_ID'], how='left')
    fourth_join.BANK_ACCOUNT_NUMBER = fourth_join.BANK_ACCOUNT_NUMBER.astype(np.int64)
    print(fourth_join.shape)
    fifth_join  = fourth_join.merge(OD_table[['BANK_ACCOUNT_NUMBER','OVERDRAFT_LIMIT']], on=['BANK_ACCOUNT_NUMBER'], how='left')
    print(fifth_join.shape)
    sixth_join   = fifth_join.merge(last_balance, on=['BANK_ACCOUNT_ID'], how='left')
    print(sixth_join.shape)
    seventh_join = sixth_join.merge(money_in_week1, on=['BANK_ACCOUNT_ID'], how='left')
    eighth_join = seventh_join.merge(money_in_week2, on=['BANK_ACCOUNT_ID'], how='left')
    ninth_join = eighth_join.merge(money_in_week3, on=['BANK_ACCOUNT_ID'], how='left')
    tenth_join = ninth_join.merge(money_in_week4, on=['BANK_ACCOUNT_ID'], how='left')
    table_rej   = tenth_join.merge(mcc_table, on=['BANK_ACCOUNT_ID'], how='left')
    print(table_rej.shape)
    if i!=len(orig_dates):
        table_rej = table_rej.merge(labling, on=['BANK_ACCOUNT_NUMBER'], how='right')
        print(table_rej.shape)
    else:
        table_rej[~table_rej['BANK_ACCOUNT_NUMBER'].isin(rel_prns)]
        print(table_rej.shape)
    table_rej['DID_NONE'] = (1 - table_rej[
        ['DID_PAYROLL', 'DID_MARKETPLACE', 'DID_FINANCIAL_INSTITUTION', 'DID_UNEMPLOYMENT', 'DID_TAX_REFUND']].max(
        axis=1))

    table_rej['TIME_FROM_LAST_TRANSACTION'] = \
    (pd.to_datetime(table_rej['PERIOD_END'], format="%Y%m%") - table_rej['LAST_TRANSACTION_IN_TIME']) \
        .astype('timedelta64[D]') 
    table_rej['TIME_FROM_LAST_MONEY_IN'] = \
    (pd.to_datetime(table_rej['PERIOD_END'], format="%Y%m%") - table_rej['LAST_TRANSACTION_MONEY_IN_IN_TIME']) \
        .astype('timedelta64[D]')
    #table_rej = table_rej.drop(['SIGNUP_DATE', 'CUSTOMER_ID', 'BANK_ACCOUNT_ID', 'PERIOD_END', 'LAST_TRANSACTION_IN_TIME', \
    #                            'LAST_TRANSACTION_MONEY_IN_IN_TIME'], axis=1)
    table_rej['AVG_MONEY_IN'] = np.where(table_rej['TOTAL_MONEY_IN'] == 0.00, 0.00, 
                                     table_rej['TOTAL_MONEY_IN'] / table_rej['TOTAL_MONEY_IN_COUNT'])

    #table_rej = table_rej.drop(['SIGNUP_DATE', 'CUSTOMER_ID', 'BANK_ACCOUNT_ID', 'PERIOD_END', 'LAST_TRANSACTION_IN_TIME', \
    #                        'LAST_TRANSACTION_MONEY_IN_IN_TIME'], axis=1)
    table_rej.fillna(0)
    table_rej= table_rej.replace(np.nan,0)
    print(table_rej.shape)
    table_rej = table_rej[table_rej['PRO_CUSTOMER']==1]
    print(table_rej.shape)
    table_rej = table_rej.drop(['PRO_CUSTOMER'], axis=1)
    table_rej.reset_index(drop = True, inplace = True)
    table_rej_all.append(table_rej)


## Concatenating dataframes together

In [ ]:
temp = table_rej_all[0]

for i in range(1, len(table_rej_all)):
        print(temp.shape)
        temp = pd.concat([temp, table_rej_all[i]], ignore_index=True)
print(temp.shape)

temp = temp.replace(np.nan,0)

# Adding the label

rejected_users_cols = '''
select * from "LILI_ANALYTICS_DEV"."ODS"."LAST_7_DAYS_OVERDRAFT_USAGE"
'''
rejected_users = pd.read_sql(rejected_users_cols, conn)
rejected_users['LABEL']=1
rejected_users = rejected_users[['LABEL', 'BANK_ACCOUNT_NUMBER','EXPIRATION_DATE']]
total_table = temp.merge(rejected_users, on=['BANK_ACCOUNT_NUMBER'], how='left')
total_table= total_table.replace(np.nan,0)
total_table = total_table.drop(['BANK_ACCOUNT_NUMBER'], axis=1)
total_table[total_table['EXPIRATION_DATE']!=0]

In [ ]:
list1 = ['IS_PRO_SINCE_SIGNUP', 'CURRENT_BALANCE',
 'TOTAL_MONEY_IN', 'TOTAL_MONEY_IN_COUNT', 'TOTAL_MONEY_IN_WEEK1',
 'TOTAL_MONEY_IN_COUNT_WEEK1', 'TOTAL_MONEY_IN_WEEK2',
 'TOTAL_MONEY_IN_COUNT_WEEK2', 'TOTAL_MONEY_IN_WEEK3', 'TOTAL_MONEY_IN_COUNT_WEEK3', 
 'TOTAL_MONEY_IN_WEEK4', 'TOTAL_MONEY_IN_COUNT_WEEK4', 'HAD_NEGATIVE_BALANCE',
 'ATM_SUM','SWIPE_SUM', 'SPEND_SUM','DIRECT_DEPOSIT_SUM','DIRECT_PAY_SUM','ACH_SUM','CHECK_SUM','GREENDOT_SUM',
 'CARD_DEPOSIT_SUM','AVERAGE_BALANCE','DID_PAYROLL','DID_MARKETPLACE','DID_FINANCIAL_INSTITUTION',
 'DID_UNEMPLOYMENT','DID_TAX_REFUND','DID_NONE','LOGIN_COUNT','PENDING_CHECK','DENIED_AUTH_NSF',
 'DENIED_AUTH_NSF_CNT','DENIED_AUTH','DENIED_AUTH_CNT','AUTH_EXP','AUTH_EXP_CNT',
 'DENIED_AUTH_INACTIVE_CARD','DENIED_AUTH_INACTIVE_CARD_CNT','DENIED_AUTH_INVALID_PIN',
 'DENIED_AUTH_INVALID_PIN_CNT','DENIED_AUTH_GAS','DENIED_AUTH_GAS_CNT','ACH_CREDIT_FAIL',
 'ACH_CREDIT_FAIL_CNT','ACH_CREDIT_RETURN','ACH_CREDIT_RETURN_CNT','CREATE_HOLD','CREATE_HOLD_CNT',
 'EXPIRE_HOLD','EXPIRE_HOLD_CNT','TIME_FROM_LAST_TRANSACTION','TIME_FROM_LAST_MONEY_IN','AVG_MONEY_IN',
 'LABEL']

cols = list(temp.columns)
cols.sort()
#print(cols)


mcc_list_cols = '''
select * from  "LILI_ANALYTICS_DEV"."ODS"."MCC_LIST_FOR_OVERDRAFT" order by MCC_CODE
'''
mcc_list = pd.read_sql(mcc_list_cols, conn)
#list1 = ['TOTAL_MONEY_IN', 'TOTAL_MONEY_IN_COUNT', 'HAD_NEGATIVE_BALANCE', 'ATM_SUM', 'SWIPE_SUM',\
#    'SPEND_SUM', 'DIRECT_DEPOSIT_SUM', 'DIRECT_PAY_SUM', 'ACH_SUM', 'CHECK_SUM', 'GREENDOT_SUM',\
#    'CARD_DEPOSIT_SUM', 'AVERAGE_BALANCE', 'DID_PAYROLL', 'DID_MARKETPLACE', 'DID_FINANCIAL_INSTITUTION',\
#    'DID_UNEMPLOYMENT', 'DID_TAX_REFUND', 'DID_NONE', 'LOGIN_COUNT', 'PENDING_CHECK', 'DENIED_AUTH_NSF',\
#    'DENIED_AUTH_NSF_CNT', 'DENIED_AUTH', 'DENIED_AUTH_CNT', 'AUTH_EXP', 'AUTH_EXP_CNT', \
#    'DENIED_AUTH_INACTIVE_CARD', 'DENIED_AUTH_INACTIVE_CARD_CNT', 'DENIED_AUTH_INVALID_PIN',\
#    'DENIED_AUTH_INVALID_PIN_CNT', 'DENIED_AUTH_GAS', 'DENIED_AUTH_GAS_CNT', 'ACH_CREDIT_FAIL',\
#    'ACH_CREDIT_FAIL_CNT', 'ACH_CREDIT_RETURN', 'ACH_CREDIT_RETURN_CNT', 'CREATE_HOLD', 'CREATE_HOLD_CNT',\
#    'EXPIRE_HOLD', 'EXPIRE_HOLD_CNT', 'TIME_FROM_LAST_TRANSACTION', 'TIME_FROM_LAST_MONEY_IN', 'AVG_MONEY_IN',\
#    'LABEL']

#table_ready = table_all.iloc[:, list]

num_all = -len(list1)-1
current_mccs = cols[:num_all]

missing_mcc = list(set(mcc_list['MCC_CODE']) - set(current_mccs))

for elt in missing_mcc:
    temp[elt] = 0
list2 = list(mcc_list['MCC_CODE'].iloc[0:])
listush = list1 + list2
final_table = temp[listush]
final_table

## Need to eliminate a few crucial columns

In [ ]:
#Let's modelize!

#First step - make seperate labels from features and convert to numpy arrays

# Labels are the values we want to predict
labels = np.array(final_table['LABEL'])
# Remove the labels from the features
# axis 1 refers to the columns
features = final_table.drop('LABEL', axis=1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)
features.shape

In [ ]:
# Second step - split the data into training and testing sets
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, stratify = labels, test_size=0.25)
features_res, labels_res = sm.fit_resample(train_features, train_labels)

In [ ]:
#Third step - train model


model_rej = xgb.XGBClassifier(max_depth=3, n_estimators=250, learning_rate=0.24, use_label_encoder=False)
model_rej.fit(features_res, labels_res)


        

In [ ]:
predictions = model_rej.predict(test_features)
print("Accuracy:", metrics.accuracy_score(predictions, test_labels))

In [ ]:
features

In [ ]:
count1 = 0
count2 = 0
for i in range(0,len(labels_res)):
    if labels_res[i]==0:
        count1=count1+1
    else:
        count2=count2+1
        
print(count1)
print(count2)

# Get numerical feature importances
importances = list(model_rej.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation='vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance');
plt.xlabel('Variable');
plt.title('Variable Importances');

In [ ]:
total_table['CURRENT_BALANCE'].hist()

In [ ]:
total_table['DAILY_BALANCE'] = total_table['DAILY_BALANCE'].astype(float)

In [ ]:
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(test_labels, predictions), cmap="Blues", annot=True, fmt=".2f")
print(metrics.classification_report(test_labels, predictions))

In [ ]:
probes = []
count = 0
ranger = np.arange(0.200,0.501,0.005)
predictions2 = model_rej.predict(test_features)
preds_proba = model_rej.predict_proba(test_features)
confusion_list = []

for elt in ranger:
    predictions2 = model_rej.predict(test_features)
    count_true_positive  = 0
    count_true_negative  = 0
    count_false_positive = 0
    count_false_negative = 0
    
    for i in range(0, len(predictions)):
        if predictions[i]==0 and preds_proba[i][1]>elt:
            predictions2[i] = 1
            
        if predictions2[i]==0 and test_labels[i]==0:
            count_true_negative += 1
            
        if predictions2[i]==0 and test_labels[i]==1:
            count_false_negative += 1
            
        if predictions2[i]==1 and test_labels[i]==1:
            count_true_positive += 1
            
        if predictions2[i]==1 and test_labels[i]==0:
            count_false_positive += 1

#     print("for value " + str(elt) + ", here are the values:")
# #    print("true negative: " + str(count_true_negative))
# #    print("true positive: " + str(count_true_positive))
#     print("false negative: " + str(count_false_negative))
#     print("false negative ratio: " + str(count_false_negative/(count_false_negative + count_true_positive)))
#     print("false positive: " + str(count_false_positive))
#     print("false positive ratio: " + str(count_false_positive/(count_false_positive + count_true_negative)))
    confusion_list.append([elt, 
                          count_false_negative/(count_false_negative + count_true_positive),
                          count_false_positive/(count_false_positive + count_true_negative)])
tf_preditions = pd.DataFrame(confusion_list, columns=['Threshold', 'false_negative_ratio',
                                     'false_positive_ratio'])
pd.set_option('display.max_rows', None)
display(tf_preditions)
    #if predictions[i]==0 and preds_proba[i][1]>elt:
    #     predictions2[i] = 1
        
#false positive - I say yes but it is false
#false negative - I say no but it is true

#count = 0
#for i in range(0, len(predictions)):
#    if test_labels[i]==1:
#        if predictions[i]==0:
#            count=count+1
#print(count)


In [ ]:
tf_preditions.plot.line(x='Threshold', y='false_negative_ratio')

In [ ]:
tf_preditions.plot.line(x='Threshold', y='false_positive_ratio')

In [ ]:
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(test_labels, predictions2), cmap="Blues", annot=True, fmt=".2f")
print(metrics.classification_report(test_labels, predictions))

In [ ]:
# Get numerical feature importances
importances = list(model_rej.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation='vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance');
plt.xlabel('Variable');
plt.title('Variable Importances');

In [ ]:
test_features[1]

In [ ]:
feature_list[46]
feature_list[2]
features_res.shape

In [ ]:
model_rej_wrapper = RejectionModel(model_reject = model_rej, threshold = REJECTION_PREDICTION_THRESHOLD)
#print(model_rej_wrapper.predict(features_res))
rej_explainer = shap.KernelExplainer(model_rej_wrapper.predict, features_res)

In [ ]:
import datetime
#saving the ODmodel
odmodel = ODModel(model_money_in=model_overdraft, model_reject=model_rej, explainer=explainer, rej_explainer = rej_explainer)
filename = f'../../model_dumps/ODmodel_{datetime.datetime.utcnow().isoformat()}.pkl'
pickle.dump(odmodel, open(filename, 'wb'))
print(filename)

In [ ]:
modeler = pickle.load(open('../../model_dumps/ODmodel_2021-10-12T06:44:43.063140.pkl', 'rb'))

In [ ]:
modeler.rej_explainer

In [ ]:
overdraft_cols = '''
select *  from "LILI_ANALYTICS_DEV"."ODS"."OVERDRAFT_LIMITS"
'''
    
overdraft = pd.read_sql(overdraft_cols, conn)


overdraft.drop(['BANK_ACCOUNT_ID'], axis=1, inplace=True)

bank_account_num_cols = '''
select distinct BANK_ACCOUNT_ID, BANK_ACCOUNT_NUMBER FROM "LILI_ANALYTICS"."ODS"."MYSQL_DW_CUSTOMER_MONTHLY_NEW"
'''



bank_account_num = pd.read_sql(bank_account_num_cols, conn)
bank_account_num.BANK_ACCOUNT_NUMBER = bank_account_num.BANK_ACCOUNT_NUMBER.astype(np.int64)

#print(bank_account_num.dtypes)
#print(overdraft.dtypes)


transaction_cols = '''
select BANK_ACCOUNT_ID, max(TRANSACTION_DATE) as max_date from "LILI_ANALYTICS"."ODS"."MYSQL_ACCOUNT_TRANSACTION_ALL"
group by BANK_ACCOUNT_ID

'''

transaction = pd.read_sql(transaction_cols, conn)

transaction['TIME_FROM_LAST_TRANSACTION'] = \
    (pd.to_datetime('2021-08-30', format="%Y%m%") - transaction['MAX_DATE']) \
        .astype('timedelta64[D]') 


balance_cols ='''
select t.BANK_ACCOUNT_ID, t.CURRENT_BALANCE
        FROM "LILI_ANALYTICS_DEV"."ODS"."MYSQL_CUSTOMER_BALANCE_HISTORY" t
        INNER JOIN (
        SELECT BANK_ACCOUNT_ID, MAX(VALID_DATE) AS MAXDATE
        FROM "LILI_ANALYTICS_DEV"."ODS"."MYSQL_CUSTOMER_BALANCE_HISTORY" tm
        GROUP BY BANK_ACCOUNT_ID
        ) tm ON t.BANK_ACCOUNT_ID = tm.BANK_ACCOUNT_ID AND t.VALID_DATE = tm.MAXDATE
'''

balance = pd.read_sql(balance_cols, conn)

od_all = overdraft.merge(bank_account_num, on=['BANK_ACCOUNT_NUMBER'], how='left')
od_all = od_all[od_all.BANK_ACCOUNT_ID.notnull()].reset_index(drop=True)
od_all = od_all.merge(balance, on=['BANK_ACCOUNT_ID'], how='left')
od_all = od_all.merge(transaction, on=['BANK_ACCOUNT_ID'], how='left')
od_all

In [ ]:
od_all[od_all['OVERDRAFT_LIMIT']!='0.0000']

In [ ]:
reasonizer = od_all[(((od_all['CURRENT_BALANCE'] < -42.5) & (od_all['OVERDRAFT_LIMIT'] == '50.0000'))^
       ((od_all['CURRENT_BALANCE'] < -34) & (od_all['OVERDRAFT_LIMIT'] == '40.0000'))^
       ((od_all['CURRENT_BALANCE'] < -17) & (od_all['OVERDRAFT_LIMIT'] == '20.0000'))) &
      (od_all['TIME_FROM_LAST_TRANSACTION'] >= 7.000) & (od_all['MODEL_REASON'] == 'model rejection')]
reasonizer['MODEL_REASON'].value_counts()

In [ ]:
od_all[od_all['MODEL_REASON']=="model rejection"]

In [ ]:
od_all[(od_all['MODEL_REASON'] == 'model rejection') & (od_all['OVERDRAFT_LIMIT'] == '50.0000')]

# MISC model check

In [ ]:
today = pd.read_csv('../../model_dumps/041021.csv', header=None)
today = today.rename(columns={0: "BANK_ACCOUNT_NUMBER", 1: "raw_overdraft", 2: "raw_expiration", 3: "raw_reason"})
today

prev = pd.read_csv('../../model_dumps/prev.csv')


today_prod_cols = '''
select * from "LILI_ANALYTICS"."ODS"."OVERDRAFT_LIMITS"
'''

today_prod = pd.read_sql(today_prod_cols, conn)
today_merged = today.merge(today_prod[['BANK_ACCOUNT_NUMBER', 'OVERDRAFT_LIMIT', 'EXPIRATION_DATE']], on=['BANK_ACCOUNT_NUMBER'], how='left')
today_merged[(today_merged['EXPIRATION_DATE'] == '2021-11-03') & (today_merged['OVERDRAFT_LIMIT'] != '0.0000')] 

In [ ]:
prev = prev.rename(columns={"bank_account_number": "BANK_ACCOUNT_NUMBER", "expiration_date": "prev_expiration"})
prev = prev[['BANK_ACCOUNT_NUMBER', 'prev_expiration']]
total_merged = today_merged.merge(prev, on=['BANK_ACCOUNT_NUMBER'], how='left')
#total_merged[((total_merged['raw_overdraft']==50) & (total_merged['prev_expiration']=='nan'))]
#total_merged[((pd.isna(total_merged['prev_expiration'])) & (total_merged['OVERDRAFT_LIMIT']!='0.0000'))]
#total_merged[((pd.isna(total_merged['prev_expiration'])) & (total_merged['raw_overdraft']!=0))]
total_merged[((pd.isna(total_merged['prev_expiration']) & (total_merged['raw_overdraft']!=0)))]


In [ ]:
quer_cols = '''
select * from "LILI_ANALYTICS"."DWH"."ML_OVERDRAFT_POST_PREDICTION"
where PREDICTION > 0
'''

quer = pd.read_sql(quer_cols, conn)
quer

In [ ]:
quer['REASON'].value_counts()

In [ ]:
quer2_cols = '''
select bank_account_id,
(CASE WHEN count(bank_account_id)>0 THEN 1 END) as IS_RECURRENT_DD
from LILI_ANALYTICS.ods.MYSQL_DW_CUSTOMER_MONTHLY_NEW dcmn
where bank_account_id in (
select bank_account_id from (
select c.bank_account_number,ata2.bank_account_id,max(total_direct_deposit) as total_direct_deposit,
SUM(CASE WHEN (ata2.act_type='PM' AND ata2.type<>'C2') OR (ata2.act_type='AD' AND (ata2.details='Debit Card transfer' OR ata2.type='FM'))
THEN ata2.transaction_amount ELSE NULL END) Total_money_in
from LILI_ANALYTICS.ods.MYSQL_ACCOUNT_TRANSACTION_ALL as ata2
inner join (
select bank_account_number,b.bank_account_id, sum(Total_Amount) as total_direct_deposit from
(select bank_account_id,bank_account_number
from LILI_ANALYTICS.ods.MYSQL_DW_CUSTOMER_MONTHLY_NEW where account_legit =1 and pro_customer=1 group by 1,2) as a
inner join (select bank_account_id,
case when dss.type is null then 'N/A' else upper(dss.type) end AS Category,
sum(transaction_amount) AS Total_Amount
from  LILI_ANALYTICS.ods.MYSQL_ACCOUNT_TRANSACTION_ALL AS ata
left join LILI_ANALYTICS.ods.MYSQL_DIRECT_DEPOSIT_SOURCES as dss on dss.merchant=ata.details
where ata.act_type='PM' and ata.type='FM'
And transaction_date >= '2021-08-29' AND transaction_date <= '2021-09-29'
group by bank_account_id, dss.type
having Category in ('PAYROLL', 'PAYMENTS', 'UNEMPLOYMENT', 'MARKETPLACE','SOCIAL SECURITY ADMINISTRATION','FEDERAL PAYROLL')
) as b
on a.bank_account_id=b.bank_account_id
group by 1,2
order by 3 desc) as c
on ata2.bank_account_id = c.bank_account_id
where transaction_date >='2021-08-29' AND transaction_date <= '2021-09-29'
group by 1,2) as d
where total_direct_deposit>=500 OR (total_money_in>=200 and total_direct_deposit>=300))
group by 1
'''


quer2 = pd.read_sql(quer2_cols, conn)
quer2

In [ ]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
df = pd.read_csv('../../model_dumps/checkme.csv', header=None)
df

In [ ]:
df = pd.read_csv('../../model_dumps/prediction0919.csv', header=None)
df = df.rename(columns={0: 'PRN', 1: 'PREDICTION', 2: 'EXPIRATION', 3: 'REASON'})
odlim_cols = '''
select BANK_ACCOUNT_NUMBER as PRN, OVERDRAFT_LIMIT as OLD_LIMIT, MODEL_REASON as OLD_REASON
from "LILI_ANALYTICS_DEV"."ODS"."OVERDRAFT_LIMITS" order by EXPIRATION_DATE desc
'''
df2 = pd.read_sql(odlim_cols, conn)
tbl_all = df.merge(df2, on=['PRN'], how='left')
tbl_all

In [ ]:
tbl_all[(tbl_all['PREDICTION']==50) & (tbl_all['OLD_LIMIT']!='50.0000')]

In [ ]:
tbl_all[(tbl_all['REASON']=='model rejection')]

In [ ]:
tbl_all[(tbl_all['REASON']=='model rejection') & (tbl_all['OLD_LIMIT']=='50.0000')]

In [ ]:
listush = [260101016723,260101019164,260101030880,260101045532,260101052470,260101055739,260101082782,260101098911,260101116044,260101127231,260101142669,260101148039,260101156545,260101170314,260101191567,260101196178,260101210136,260101228880,260101244853,260101262202,260101280071,260101298487,260101322642,260101327161,260101347722,260101349256,260101363448,260101366615,260101387025,260101393627,260101398311,260101399889,260101399897,260101404192,260101412161,260101416808,260101419216,260101424349,260101426229,260101432623,260101433381,260101437796,260101451961,260101456705,260101488179,260101488641,260101489722,260101490878,260101493781,260101497097,260101497808,260101510568,260101510998,260101511251,260101529857,260101533560,260101537611,260101542561,260101548592,260101549657,260101554061,260101557825,260101560316,260101562502,260101569010,260101573053,260101574648,260101575298,260101576700,260101584837,260101585396,260101589687,260101596054,260101602977,260101611663,260101614154,260101614709,260101617843,260101631638,260101635621,260101640894,260101642379,260101646545,260101646875,260101647162,260101655330,260101656155,260101664852,260101670370,260101671162,260101703916,260101709079,260101721199,260101745313,260101745479,260101754638,260101757540,260101761260,260101763050,260101769636,260101770303,260101771111,260101772192,260101780278,260101780302,260101781862,260101785152,260101787513,260101796217,260101803104,260101808335,260101815355,260101827087,260101828580,260101835924,260101842383,260101844504,260101856722,260101860039,260101872620,260101875557,260101890085,260101891893,260101898716,260101906352,260101915528,260101923704,260101933125,260101936946,260101938553,260101955532,260101967818,260101969400,260101975282,260101989937,260101999282,260102006186,260102008497,260102008927,260102011335,260102018413,260102018975,260102024148,260102042272,260102044294,260102047263,260102053949,260102056074,260102056280,260102058724,260102059367,260102065596,260102065778,260102076270,260102077898,260102083870,260102085677,260102086907,260102094117,260102100542,260102109303,260102123718,260102134079,260102134483,260102144623,260102146586,260102147196,260102151040,260102152949,260102157724,260102159225,260102191228,260102195344,260102204898,260102207743,260102207990,260102210457,260102223278,260102225034,260102235058,260102237443,260102248507,260102252996,260102265261,260102307188,260102328937,260102331048,260102338639,260102341195,260102345378,260102352218,260102371333,260102379823,260102382413,260102387263,260102403144,260102403219,260102409299,260102412996,260102414786,260102430295,260102431657,260102436938,260102459393,260102464369,260102464393,260102465341,260102466794,260102470580,260102475290,260102494739,260102496478,260102497369,260102528585,260102530664,260102536026,260102538774,260102539509,260102543469,260102544707,260102557030,260102557485,260102559176,260102572765,260102575453,260102581295,260102586864,260102587136,260102589678,260102593613,260102605151,260102607736,260102609112,260102610409,260102612009,260102612868,260102615663,260102620622,260102629839,260102631231,260102645546,260102648243,260102649480,260102651254,260102654357,260102654456,260102656345,260102664737,260102673324,260102677812,260102682283,260102684719,260102686136,260102688728,260102689452,260102702362,260102706744,260102709722,260102711470,260102712015,260102712957,260102715257,260102716347,260102723277,260102730322,260102735792,260102738978,260102741105,260102741352,260102741618,260102746906,260102756236,260102761350,260102764115,260102764842,260102767969,260102775970,260102777802,260102782166,260102784881,260102786985,260102796406,260102811015,260102822996,260102825593,260102834678,260102841715,260102844230,260102845880,260102847142,260102852399,260102853512,260102856051,260102860061,260102860764,260102861796,260102869427,260102878626,260102879376,260102880325,260102881216,260102893708,260102894193,260102894482,260102895588,260102896727,260102896818,260102897436,260102901022,260102903242,260102904257,260102908910,260102920758,260102922234,260102925906,260102927936,260102933967,260102935152,260102935665,260102937646,260102940749,260102944782,260102950987,260102958824,260102969052,260102969227,260102976008,260102976552,260102985454,260102988862,260102992146,260102999356,260103007449,260103013207,260103016051,260103020848,260103030961,260103034484,260103038394,260103042271,260103050696,260103053625,260103057303,260103065256,260103065827,260103067997,260103070389,260103077293,260103079406,260103080230,260103080792,260103081824,260103084158,260103087045,260103089124,260103089983,260103093001,260103093860,260103094009,260103097788,260103103453,260103107918,260103110615,260103118642,260103123246,260103128765,260103134425,260103139218,260103141008,260103143533,260103148508,260103150611,260103151759,260103157566,260103157806,260103161147,260103162673,260103173456,260103175758,260103176095,260103182010,260103186631,260103188900,260103188991,260103189221,260103191615,260103195780,260103198115,260103198909,260103205662,260103209011,260103209912,260103210647,260103210936,260103211025,260103221248,260103223459,260103226858,260103234050,260103237087,260103237681,260103237988,260103241949,260103243390,260103248258,260103260634,260103262895,260103265419,260103265427,260103266730,260103267712,260103268140,260103271045,260103272167,260103277505,260103280988,260103282505,260103283297,260103289997,260103292975,260103293866,260103301289,260103301339,260103304457,260103305041,260103307070,260103309514,260103317863,260103319406,260103325353,260103329231,260103330106,260103332144,260103335451,260103336327,260103340774,260103348033,260103354288,260103354999,260103356002,260103359832,260103373593,260103379780,260103384699,260103386470,260103389102,260103395547,260103397675,260103398392,260103400768,260103402368,260103405452,260103406328,260103408100,260103408514,260103413860,260103417127,260103425278,260103426037,260103426565,260103429692,260103432092,260103436945,260103439691,260103447058,260103451738,260103451977,260103453825,260103455358,260103456737,260103458089,260103458907,260103459855,260103462230,260103468567,260103470423,260103472205,260103472759,260103479002,260103479085,260103483137,260103485538,260103487575,260103491189,260103496592,260103502431,260103504478,260103504981,260103505632,260103514576,260103518254,260103518460,260103521019,260103526729,260103538658,260103541306,260103542734,260103544771,260103545703,260103548814,260103550547,260103551339,260103553368,260103553475,260103554051,260103557989,260103560777,260103565123,260103567525,260103577904,260103581005,260103584439,260103586145,260103586467,260103590253,260103590642,260103592283,260103594362,260103596292,260103598280,260103599858,260103604393,260103604799,260103606208,260103607107,260103616355,260103625117,260103626222,260103628954,260103631198,260103638979,260103641379,260103648135,260103648218,260103649257,260103651386,260103656997,260103659173,260103667499,260103668398,260103671160,260103671756,260103672424,260103675526,260103679080,260103679874,260103685079,260103688180,260103690541,260103692463,260103693768,260103696092,260103698254,260103704623,260103707816,260103708251,260103711263,260103713665,260103720280,260103726683,260103749677,260103750576,260103757118,260103757365,260103766440,260103768149,260103772570,260103774154,260103776571,260103793972,260103794178,260103795019,260103797387,260103802609,260103806774,260103806956,260103808663,260103813499,260103821872,260103821955,260103823423,260103824538,260103824884,260103842183,260103845145,260103847737,260103851119,260103855425,260103856225,260103857298,260103858858,260103860524,260103861779,260103861795,260103862413,260103869400,260103871968,260103873329,260103874699,260103875076,260103877338,260103880050,260103882023,260103882445,260103883732,260103885190,260103887022,260103887238,260103889606,260103889622,260103890471,260103892709,260103894291,260103901393,260103914198,260103915666,260103917092,260103920526,260103922761,260103924858,260103927539,260103928206,260103929634,260103939609,260103939740,260103941316,260103947388,260103948832,260103949756,260103950655,260103951752,260103953303,260103957395,260103958369,260103961462,260103964284,260103969002,260103972972,260103974143,260103976346,260103980819,260103983045,260103986659,260103988135,260103989851,260103991873,260103992988,260103993242,260103998316,260104005095,260104005533,260104006309,260104010269,260104010541,260104011671,260104012182,260104012398,260104012844,260104013081,260104013586,260104017835,260104019005,260104021381,260104022066,260104022488,260104025424,260104027222,260104029756,260104035092,260104042734,260104043393,260104047691,260104047873,260104050604,260104060330,260104071931,260104076690,260104078589,260104079140,260104079215,260104079843,260104080676,260104081310,260104082300,260104090097,260104091061,260104102421,260104108030,260104108501,260104112313,260104113758,260104124078,260104200902,260104371158,260104413729,260104565577]
trouble = df[df['PRN'].isin(listush)]
trouble

In [ ]:
trouble[trouble['PREDICTION']==50]

In [ ]:
prn_cols = '''SELECT BANK_ACCOUNT_ID,
        MAX(BANK_ACCOUNT_NUMBER) AS BANK_ACCOUNT_NUMBER
        FROM "LILI_ANALYTICS"."ODS"."MYSQL_DW_CUSTOMER_MONTHLY_NEW"
GROUP BY BANK_ACCOUNT_ID
'''

prn = pd.read_sql(prn_cols, conn)

prn = prn.rename(columns={'BANK_ACCOUNT_NUMBER': 'PRN'})

prn

In [ ]:
prn['PRN'] = prn['PRN'].astype(int)
combo = df.merge(prn, on=['PRN'], how='left')
combo

In [ ]:
bads = combo[combo['PRN'].isin(listush)]
bad_ids = list(bads['BANK_ACCOUNT_ID'])

In [ ]:
print("[", end =" "),
for elt in bad_ids:
    print(elt, end =" "),
    print(",", end =" ")
print("]")

In [ ]:
bad_ids = [ 100166 , 100182 , 101098 , 101484 , 101856 , 102035 , 102693 , 102760 , 102841 , 102949 , 103757 , 103760 , 103916 , 104245 , 104481 , 105351 , 106040 , 106597 , 107323 , 108575 , 108725 , 109459 , 110105 , 110317 , 111612 , 111943 , 113202 , 113495 , 114948 , 115129 , 115811 , 116575 , 11667 , 117492 , 118310 , 119252 , 119634 , 119795 , 121493 , 122721 , 122880 , 123468 , 124933 , 125868 , 126558 , 126789 , 126832 , 127073 , 127781 , 127898 , 128415 , 130228 , 130430 , 130727 , 131395 , 131608 , 131629 , 131873 , 131937 , 132560 , 132578 , 133628 , 133790 , 134388 , 134573 , 134696 , 135417 , 136060 , 136936 , 138448 , 139484 , 139525 , 14050 , 140539 , 140735 , 140796 , 141181 , 141371 , 141849 , 141999 , 145199 , 145611 , 146566 , 146851 , 146876 , 147122 , 148404 , 148580 , 149582 , 149821 , 150927 , 151376 , 152603 , 156903 , 15844 , 159114 , 159325 , 160084 , 160340 , 160758 , 161442 , 163443 , 164292 , 164551 , 165036 , 166714 , 166721 , 167329 , 167699 , 167878 , 169429 , 169565 , 16958 , 170093 , 172375 , 172872 , 172875 , 172970 , 173115 , 173494 , 173965 , 175909 , 176083 , 176172 , 179345 , 179553 , 180089 , 180364 , 180437 , 180833 , 180957 , 182190 , 182235 , 182404 , 183763 , 184032 , 184616 , 18500 , 185173 , 185200 , 185454 , 185848 , 187002 , 187260 , 187398 , 187527 , 187687 , 187773 , 188053 , 188544 , 189465 , 189605 , 19037 , 191036 , 191306 , 191430 , 191607 , 191917 , 191927 , 192116 , 192955 , 193814 , 194263 , 194710 , 194953 , 195095 , 195354 , 195427 , 196718 , 197156 , 197454 , 197629 , 197683 , 197777 , 198007 , 198116 , 198809 , 19888 , 199514 , 200061 , 200379 , 200592 , 200617 , 200643 , 201197 , 202130 , 202642 , 202918 , 202991 , 203304 , 204105 , 204288 , 204724 , 204996 , 205206 , 206148 , 207609 , 208807 , 209067 , 209975 , 210679 , 210931 , 211096 , 211222 , 211747 , 211859 , 212113 , 212514 , 212584 , 212687 , 213450 , 21352 , 214370 , 214445 , 214540 , 214629 , 215878 , 215927 , 215956 , 216066 , 216180 , 216189 , 216251 , 216610 , 216832 , 216933 , 217399 , 218656 , 218804 , 219171 , 219374 , 219977 , 220096 , 220147 , 220345 , 220655 , 221059 , 221679 , 222463 , 223486 , 223503 , 224181 , 224236 , 225126 , 225467 , 225795 , 226516 , 227325 , 227901 , 228186 , 228665 , 229677 , 230029 , 230420 , 230808 , 231650 , 231943 , 232311 , 233106 , 233163 , 233380 , 233619 , 234310 , 234521 , 234604 , 234660 , 234763 , 23477 , 234996 , 235285 , 235493 , 235579 , 235881 , 235967 , 235981 , 236359 , 236926 , 237372 , 237642 , 238445 , 238905 , 23938 , 239457 , 240023 , 240502 , 240681 , 240934 , 241431 , 241642 , 241756 , 242337 , 242361 , 242695 , 242848 , 243926 , 244156 , 244190 , 244782 , 245244 , 245471 , 245480 , 245503 , 245742 , 246159 , 246392 , 246471 , 247147 , 247482 , 247572 , 247645 , 247674 , 247683 , 248759 , 248980 , 249320 , 250040 , 250343 , 250403 , 250433 , 250829 , 250974 , 251460 , 252698 , 252924 , 253176 , 253177 , 253308 , 253406 , 253449 , 253739 , 253851 , 25431 , 254385 , 254733 , 254885 , 254964 , 255634 , 255932 , 256021 , 256763 , 256768 , 257084 , 257140 , 257212 , 257415 , 257659 , 258494 , 259342 , 259730 , 259817 , 260021 , 260352 , 260439 , 260884 , 261610 , 262235 , 262306 , 262407 , 262790 , 264166 , 264785 , 265276 , 265454 , 265717 , 266361 , 266574 , 266646 , 266883 , 267043 , 267352 , 267439 , 267617 , 267658 , 268193 , 268519 , 269334 , 269410 , 269463 , 269776 , 270016 , 270501 , 270776 , 271512 , 271980 , 272004 , 272189 , 272342 , 272480 , 272615 , 272697 , 272792 , 273030 , 273663 , 273849 , 274027 , 27405 , 274082 , 274707 , 274715 , 275120 , 275360 , 275564 , 275925 , 276466 , 277050 , 277254 , 277305 , 277370 , 278264 , 278632 , 278653 , 278908 , 279479 , 280672 , 280937 , 281080 , 281284 , 281377 , 281688 , 281861 , 281940 , 282143 , 282154 , 282212 , 282605 , 282884 , 283400 , 283640 , 284678 , 284988 , 285331 , 285502 , 285534 , 285913 , 285952 , 286116 , 286324 , 286517 , 286716 , 286873 , 287327 , 287367 , 287508 , 287598 , 288523 , 289399 , 289510 , 289783 , 290007 , 29001 , 290785 , 291025 , 291701 , 291709 , 291813 , 292026 , 292587 , 292805 , 293637 , 293727 , 294004 , 294063 , 294130 , 294440 , 294796 , 294875 , 295395 , 295706 , 295942 , 296134 , 296264 , 296497 , 296713 , 297350 , 297669 , 297713 , 298014 , 298254 , 298916 , 299556 , 301855 , 301945 , 302599 , 302624 , 303532 , 303702 , 304145 , 304303 , 304545 , 306285 , 306305 , 306389 , 306626 , 307148 , 30730 , 307565 , 307583 , 307754 , 308237 , 309122 , 309130 , 309277 , 309388 , 309423 , 311153 , 311449 , 311708 , 312144 , 312575 , 312655 , 312762 , 312918 , 313085 , 313210 , 313212 , 313274 , 313973 , 314229 , 314365 , 314502 , 314540 , 314766 , 315038 , 315235 , 315277 , 315406 , 315552 , 315735 , 315756 , 315993 , 315995 , 316080 , 316303 , 316462 , 317172 , 318452 , 318599 , 318742 , 319085 , 319309 , 319518 , 319786 , 319853 , 319996 , 320993 , 321007 , 321164 , 321724 , 321869 , 321961 , 322051 , 322161 , 322316 , 322725 , 322822 , 323132 , 323414 , 323886 , 324283 , 324400 , 324620 , 325067 , 32517 , 325290 , 325651 , 325799 , 325971 , 326173 , 326284 , 326310 , 326817 , 327495 , 327539 , 327616 , 328012 , 328040 , 328153 , 328204 , 328225 , 328270 , 328294 , 328344 , 328769 , 328886 , 329124 , 329192 , 329234 , 329528 , 329708 , 329961 , 330495 , 331259 , 331325 , 331755 , 331773 , 332046 , 333019 , 334179 , 334655 , 334844 , 334900 , 334907 , 334970 , 335053 , 335117 , 335216 , 335995 , 336092 , 337228 , 337789 , 337836 , 338217 , 338361 , 339393 , 34456 , 347076 , 3491 , 364187 , 368444 , 36884 , 37336 , 3734 , 383629 , 39490 , 39643 , 41062 , 41370 , 43695 , 44389 , 44960 , 45117 , 45118 , 45611 , 46470 , 46955 , 47196 , 47835 , 48212 , 48911 , 48987 , 49428 , 50936 , 51567 , 5346 , 54927 , 54974 , 55082 , 55197 , 55585 , 55916 , 55987 , 57349 , 57392 , 57418 , 59595 , 60402 , 61539 , 62057 , 62936 , 63094 , 64248 , 64624 , 65088 , 66424 , 6852 , 72822 , 76213 , 77150 , 77424 , 79197 , 80108 , 80202 , 8022 , 80719 , 81526 , 82312 , 8348 , 83503 , 83926 , 84012 , 84535 , 86458 , 87008 , 87729 , 87877 , 88413 , 88446 , 88497 , 89442 , 89598 , 90909 , 91547 , 91626 , 95727 , 96243 , 97455]

In [ ]:
len(bad_ids)

In [ ]:
odlims_cols = '''select * from "LILI_ANALYTICS_DEV"."ODS"."OVERDRAFT_LIMITS"
'''

odlims = pd.read_sql(odlims_cols, conn)

odlims = odlims[odlims['BANK_ACCOUNT_NUMBER'].isin(listush)]
odlims

In [ ]:
dates = odlims['EXPIRATION_DATE'].value_counts()
dates

In [ ]:
for elt in dates.keys():
    print(f"'{elt}'")

In [ ]:
odlims[odlims['MODEL_REASON']=='model rejection']

In [ ]:
print(odlims[['BANK_ACCOUNT_NUMBER', 'OVERDRAFT_LIMIT']])

In [ ]:
Dict = dict({
0: 'TOTAL_MONEY_IN',
1: 'TOTAL_MONEY_IN_COUNT',
2: 'TOTAL_MONEY_IN_WEEK1',
3: 'TOTAL_MONEY_IN_COUNT_WEEK1',
4: 'TOTAL_MONEY_IN_WEEK2',
5: 'TOTAL_MONEY_IN_COUNT_WEEK2',
6: 'TOTAL_MONEY_IN_WEEK3',
7: 'TOTAL_MONEY_IN_COUNT_WEEK3',
8: 'TOTAL_MONEY_IN_WEEK4',
9: 'TOTAL_MONEY_IN_COUNT_WEEK4',
10: 'HAD_NEGATIVE_BALANCE',
11: 'ATM_SUM',
12: 'SWIPE_SUM',
13: 'SPEND_SUM',
14: 'DIRECT_DEPOSIT_SUM',
15: 'DIRECT_PAY_SUM',
16: 'ACH_SUM',
17: 'CHECK_SUM',
18: 'GREENDOT_SUM',
19: 'CARD_DEPOSIT_SUM',
20: 'AVERAGE_BALANCE',
21: 'DID_PAYROLL',
22: 'DID_MARKETPLACE',
23: 'DID_FINANCIAL_INSTITUTION',
24: 'DID_UNEMPLOYMENT',
25: 'DID_TAX_REFUND',
26: 'DID_NONE',
27: 'LOGIN_COUNT',
28: 'PENDING_CHECK',
29: 'DENIED_AUTH_NSF',
30: 'DENIED_AUTH_NSF_CNT',
31: 'DENIED_AUTH',
32: 'DENIED_AUTH_CNT',
33: 'AUTH_EXP',
34: 'AUTH_EXP_CNT',
35: 'DENIED_AUTH_INACTIVE_CARD',
36: 'DENIED_AUTH_INACTIVE_CARD_CNT',
37: 'DENIED_AUTH_INVALID_PIN',
38: 'DENIED_AUTH_INVALID_PIN_CNT',
39: 'DENIED_AUTH_GAS',
40: 'DENIED_AUTH_GAS_CNT',
41: 'ACH_CREDIT_FAIL',
42: 'ACH_CREDIT_FAIL_CNT',
43: 'ACH_CREDIT_RETURN',
44: 'ACH_CREDIT_RETURN_CNT',
45: 'CREATE_HOLD',
46: 'CREATE_HOLD_CNT',
47: 'EXPIRE_HOLD',
48: 'EXPIRE_HOLD_CNT',
49: 'TIME_FROM_LAST_TRANSACTION',
50: 'TIME_FROM_LAST_MONEY_IN',
51: 'AVG_MONEY_IN',
52: '0742',
53: '0763',
54: '0780',
55: '1520',
56: '1711',
57: '1731',
58: '1740',
59: '1750',
60: '1761',
61: '1771',
62: '1799',
63: '2741',
64: '2791',
65: '2842',
66: '3000',
67: '3001',
68: '3005',
69: '3007',
70: '3008',
71: '3009',
72: '3010',
73: '3022',
74: '3026',
75: '3032',
76: '3035',
77: '3037',
78: '3039',
79: '3047',
80: '3050',
81: '3058',
82: '3061',
83: '3066',
84: '3069',
85: '3076',
86: '3084',
87: '3098',
88: '3102',
89: '3132',
90: '3136',
91: '3144',
92: '3174',
93: '3175',
94: '3196',
95: '3219',
96: '3256',
97: '3260',
98: '3351',
99: '3355',
100: '3357',
101: '3359',
102: '3366',
103: '3370',
104: '3386',
105: '3387',
106: '3389',
107: '3390',
108: '3393',
109: '3395',
110: '3405',
111: '3441',
112: '3501',
113: '3502',
114: '3503',
115: '3504',
116: '3508',
117: '3509',
118: '3510',
119: '3512',
120: '3513',
121: '3515',
122: '3516',
123: '3519',
124: '3520',
125: '3523',
126: '3526',
127: '3527',
128: '3528',
129: '3530',
130: '3532',
131: '3535',
132: '3542',
133: '3543',
134: '3551',
135: '3553',
136: '3555',
137: '3558',
138: '3559',
139: '3561',
140: '3562',
141: '3563',
142: '3564',
143: '3565',
144: '3567',
145: '3575',
146: '3576',
147: '3581',
148: '3583',
149: '3584',
150: '3588',
151: '3589',
152: '3590',
153: '3591',
154: '3592',
155: '3595',
156: '3596',
157: '3597',
158: '3604',
159: '3607',
160: '3608',
161: '3609',
162: '3613',
163: '3614',
164: '3615',
165: '3617',
166: '3618',
167: '3619',
168: '3621',
169: '3626',
170: '3627',
171: '3628',
172: '3629',
173: '3631',
174: '3634',
175: '3637',
176: '3638',
177: '3640',
178: '3641',
179: '3644',
180: '3649',
181: '3650',
182: '3652',
183: '3654',
184: '3660',
185: '3662',
186: '3665',
187: '3667',
188: '3670',
189: '3671',
190: '3676',
191: '3679',
192: '3684',
193: '3685',
194: '3687',
195: '3690',
196: '3692',
197: '3693',
198: '3694',
199: '3695',
200: '3697',
201: '3700',
202: '3703',
203: '3704',
204: '3705',
205: '3706',
206: '3708',
207: '3709',
208: '3710',
209: '3715',
210: '3717',
211: '3721',
212: '3722',
213: '3726',
214: '3728',
215: '3730',
216: '3731',
217: '3734',
218: '3735',
219: '3738',
220: '3740',
221: '3741',
222: '3742',
223: '3745',
224: '3750',
225: '3751',
226: '3763',
227: '3764',
228: '3765',
229: '3769',
230: '3770',
231: '3771',
232: '3773',
233: '3774',
234: '3775',
235: '3777',
236: '3778',
237: '3779',
238: '3780',
239: '3782',
240: '3785',
241: '3786',
242: '3816',
243: '4111',
244: '4112',
245: '4119',
246: '4121',
247: '4131',
248: '4214',
249: '4215',
250: '4225',
251: '4411',
252: '4457',
253: '4468',
254: '4511',
255: '4582',
256: '4722',
257: '4784',
258: '4789',
259: '4812',
260: '4814',
261: '4816',
262: '4821',
263: '4829',
264: '4899',
265: '4900',
266: '5013',
267: '5021',
268: '5039',
269: '5044',
270: '5045',
271: '5046',
272: '5047',
273: '5051',
274: '5065',
275: '5072',
276: '5074',
277: '5085',
278: '5094',
279: '5099',
280: '5111',
281: '5122',
282: '5131',
283: '5137',
284: '5139',
285: '5169',
286: '5172',
287: '5192',
288: '5193',
289: '5198',
290: '5199',
291: '5200',
292: '5211',
293: '5231',
294: '5251',
295: '5261',
296: '5271',
297: '5300',
298: '5309',
299: '5310',
300: '5311',
301: '5331',
302: '5399',
303: '5411',
304: '5422',
305: '5441',
306: '5451',
307: '5462',
308: '5499',
309: '5511',
310: '5521',
311: '5532',
312: '5533',
313: '5541',
314: '5542',
315: '5551',
316: '5561',
317: '5571',
318: '5598',
319: '5599',
320: '5611',
321: '5621',
322: '5631',
323: '5641',
324: '5651',
325: '5655',
326: '5661',
327: '5681',
328: '5691',
329: '5697',
330: '5698',
331: '5699',
332: '5712',
333: '5713',
334: '5714',
335: '5718',
336: '5719',
337: '5722',
338: '5732',
339: '5733',
340: '5734',
341: '5735',
342: '5811',
343: '5812',
344: '5813',
345: '5814',
346: '5815',
347: '5816',
348: '5817',
349: '5818',
350: '5912',
351: '5921',
352: '5931',
353: '5932',
354: '5933',
355: '5935',
356: '5937',
357: '5940',
358: '5941',
359: '5942',
360: '5943',
361: '5944',
362: '5945',
363: '5946',
364: '5947',
365: '5948',
366: '5949',
367: '5950',
368: '5960',
369: '5962',
370: '5963',
371: '5964',
372: '5965',
373: '5966',
374: '5967',
375: '5968',
376: '5969',
377: '5970',
378: '5971',
379: '5972',
380: '5973',
381: '5975',
382: '5976',
383: '5977',
384: '5978',
385: '5983',
386: '5992',
387: '5993',
388: '5994',
389: '5995',
390: '5996',
391: '5997',
392: '5998',
393: '5999',
394: '6010',
395: '6011',
396: '6012',
397: '6051',
398: '6211',
399: '6300',
400: '6513',
401: '7011',
402: '7012',
403: '7032',
404: '7033',
405: '7210',
406: '7211',
407: '7216',
408: '7217',
409: '7221',
410: '7230',
411: '7251',
412: '7261',
413: '7273',
414: '7276',
415: '7277',
416: '7278',
417: '7296',
418: '7297',
419: '7298',
420: '7299',
421: '7311',
422: '7321',
423: '7333',
424: '7338',
425: '7339',
426: '7342',
427: '7349',
428: '7361',
429: '7372',
430: '7375',
431: '7379',
432: '7392',
433: '7393',
434: '7394',
435: '7395',
436: '7399',
437: '7512',
438: '7513',
439: '7519',
440: '7523',
441: '7531',
442: '7534',
443: '7535',
444: '7538',
445: '7542',
446: '7549',
447: '7622',
448: '7623',
449: '7629',
450: '7631',
451: '7641',
452: '7692',
453: '7699',
454: '7800',
455: '7801',
456: '7802',
457: '7829',
458: '7832',
459: '7841',
460: '7911',
461: '7922',
462: '7929',
463: '7932',
464: '7933',
465: '7941',
466: '7991',
467: '7992',
468: '7993',
469: '7994',
470: '7995',
471: '7996',
472: '7997',
473: '7998',
474: '7999',
475: '8011',
476: '8021',
477: '8041',
478: '8042',
479: '8043',
480: '8049',
481: '8050',
482: '8062',
483: '8071',
484: '8099',
485: '8111',
486: '8211',
487: '8220',
488: '8241',
489: '8244',
490: '8249',
491: '8299',
492: '8351',
493: '8398',
494: '8641',
495: '8651',
496: '8661',
497: '8675',
498: '8699',
499: '8734',
500: '8911',
501: '8931',
502: '8999',
503: '9211',
504: '9222',
505: '9223',
506: '9311',
507: '9399',
508: '9402',
509: '9405'})

In [ ]:
Dict[0]

In [ ]:
for i in range(0, len(feature_list)):
    if Dict[i]!=feature_list[i]:
        print(i)

In [ ]:
lst = [-1,-2,-3,-4]

lst = np.abs(lst)
lst

In [ ]:
df = pd.read_csv('../../model_dumps/131021.csv', header=None)
lst = df[3].value_counts()
for i in range(0, lst.keys().shape[0]):
    print(lst.keys()[i] + "- amount: " + str(lst[i]))

In [ ]:
lst = [[1,2,1],[3,5,3], [6,4,7]]
for elt in lst:
    print(elt[1])

In [ ]:
df = pd.read_csv('../../model_dumps/integ201021.csv', header=None)
df = df.rename(columns={0: "BANK_ACCOUNT_NUMBER", 1: "NEW_LIMIT", 2: "NEW_EXP", 3: "NEW_REASON"})
df

In [ ]:
df[(df['NEW_LIMIT']==0) & (df['NEW_REASON']!='not recurrent and not close expiration')]

In [ ]:
OD_cols = ''' select *  from "LILI_ANALYTICS_DEV"."ODS"."OVERDRAFT_LIMITS" '''

OD_table = pd.read_sql(OD_cols, conn)
OD_table

In [ ]:
merged = df.merge(OD_table, on=['BANK_ACCOUNT_NUMBER'], how='left')
merged

In [ ]:
prns = list(merged[(merged['NEW_LIMIT']==0) & (merged['OVERDRAFT_LIMIT']!='0.0000') & (merged['NEW_REASON']!='not recurrent and not close expiration')]['BANK_ACCOUNT_NUMBER'])

In [ ]:
prns

In [ ]:
lst = [260101892867, 260101919967, 260101978336, 260102095411, 260102159118, 260102374618, 260102564309, 260102743085, 260103052189, 260103381257, 260103620860, 260104063136, 260104070503, 260104217765, 260104274386, 260104274501, 260104388848, 260104492277, 260104730023, 260101419950, 260101029817, 260101509727, 260101519791, 260102289709]
df[df[0].isin(lst)]

In [ ]:
querush  =   '''select * from 
(SELECT BANK_ACCOUNT_ID,
        MAX(BANK_ACCOUNT_NUMBER) AS BANK_ACCOUNT_NUMBER,
        MAX((CASE WHEN PRODUCT_ID = 20643 THEN 1 ELSE 0 END)) AS IS_PRO_SINCE_SIGNUP
FROM "LILI_ANALYTICS"."ODS"."MYSQL_DW_CUSTOMER_MONTHLY_NEW"
WHERE BANK_ACCOUNT_NUMBER in (260101751642,
 260101775336,
 260101796415,
 260101846491,
 260101848976,
 260101857829,
 260101878239,
 260101878262,
 260101900116,
 260101082295,
 260101906618,
 260101925519,
 260101947430,
 260101983468,
 260102062650,
 260102068384,
 260102085917,
 260102125929,
 260102138195,
 260102146917,
 260102170271,
 260102180494,
 260102192465,
 260102202561,
 260102204351,
 260102204740,
 260102207057,
 260102216686,
 260102234135,
 260102235124,
 260102316049,
 260102364114,
 260102365889,
 260101121671,
 260102382413,
 260102402534,
 260102413739,
 260102443231,
 260102446556,
 260102490430,
 260102504677,
 260102508264,
 260102525763,
 260101138204,
 260102544400,
 260102554698,
 260102571916,
 260102573441,
 260102577582,
 260102608890,
 260102612009,
 260102614492,
 260102640398,
 260102645025,
 260102647104,
 260102647740,
 260102657509,
 260102666666,
 260102674462,
 260102705605,
 260102707932,
 260102729837,
 260102764446,
 260102773355,
 260102796513,
 260102825452,
 260102827151,
 260102842671,
 260102875457,
 260102898202,
 260102907227,
 260102913134,
 260102934395,
 260102963089,
 260102974938,
 260102978038,
 260102981412,
 260102994399,
 260103000873,
 260103010674,
 260103013645,
 260103032397,
 260103062584,
 260103068243,
 260101190379,
 260103090502,
 260103097390,
 260103103925,
 260103105219,
 260103113841,
 260103126371,
 260103128765,
 260103167789,
 260103184966,
 260103191003,
 260103203154,
 260103235073,
 260103251633,
 260103256525,
 260103272662,
 260103298378,
 260103329082,
 260103341780,
 260103350088,
 260103353132,
 260103364766,
 260103372207,
 260103376992,
 260103378204,
 260103379913,
 260103381992,
 260103385522,
 260103399036,
 260103408373,
 260103410155,
 260103414470,
 260103444121,
 260103451555,
 260103455697,
 260103474664,
 260103482303,
 260103484333,
 260103488730,
 260103515219,
 260103516530,
 260103523288,
 260103525002,
 260103529129,
 260103531174,
 260103541637,
 260103552162,
 260103558789,
 260103570958,
 260103574331,
 260103575403,
 260101239440,
 260103587432,
 260103624334,
 260103648499,
 260103658381,
 260103682290,
 260103702379,
 260103708228,
 260103709184,
 260103740098,
 260103744504,
 260103745410,
 260103767497,
 260103806774,
 260103823316,
 260103838793,
 260103861357,
 260103870762,
 260103871968,
 260103876728,
 260103878542,
 260103933198,
 260103961975,
 260103964367,
 260104003603,
 260104012042,
 260104038542,
 260104043401,
 260104044086,
 260104045620,
 260104058433,
 260104072376,
 260104077698,
 260104079876,
 260101289742,
 260104090493,
 260104091616,
 260104116447,
 260104133376,
 260104144100,
 260104155510,
 260104162607,
 260104176185,
 260104188552,
 260104194964,
 260104200605,
 260104200936,
 260104218524,
 260104226790,
 260104233960,
 260104245097,
 260104248661,
 260104283866,
 260104289939,
 260104300017,
 260104305156,
 260104323654,
 260104328158,
 260104332143,
 260104355896,
 260104375647,
 260104404694,
 260104436662,
 260104450093,
 260101326916,
 260104462841,
 260104464250,
 260104464946,
 260104465794,
 260104468632,
 260101327914,
 260104474952,
 260104479621,
 260101330744,
 260104535117,
 260104536172,
 260104543202,
 260104546924,
 260104564620,
 260104564877,
 260104592407,
 260104594346,
 260104622527,
 260104632435,
 260104636683,
 260104638986,
 260104640883,
 260104657580,
 260104672191,
 260104676010,
 260104686910,
 260104704846,
 260104750849,
 260104783121,
 260104813639,
 260104916598,
 260101371433,
 260101427854,
 260101442598,
 260101447399,
 260101447779,
 260101464832,
 260101471621,
 260101494466,
 260101496560,
 260101503118,
 260101516276,
 260101517118,
 260101552412,
 260101558799,
 260101562635,
 260101569994,
 260101581213,
 260101585016,
 260101594026,
 260101638914,
 260101649465,                        
 260101660785,
 260101662997,
 260101690550,
 260101709111)
GROUP BY BANK_ACCOUNT_ID) bank
left join(

    SELECT t.BANK_ACCOUNT_ID, t.CURRENT_BALANCE
FROM "LILI_ANALYTICS_DEV"."ODS"."MYSQL_CUSTOMER_BALANCE_HISTORY" t
INNER JOIN (
SELECT BANK_ACCOUNT_ID, MAX(CASE WHEN VALID_DATE <=  '2021-10-20' THEN VALID_DATE END) AS MAXDATE
FROM "LILI_ANALYTICS_DEV"."ODS"."MYSQL_CUSTOMER_BALANCE_HISTORY" tm
GROUP BY BANK_ACCOUNT_ID
) tm ON t.BANK_ACCOUNT_ID = tm.BANK_ACCOUNT_ID AND t.VALID_DATE = tm.MAXDATE) dm on bank.BANK_ACCOUNT_ID = dm.BANK_ACCOUNT_ID'''

table = pd.read_sql(querush, conn)
table

In [ ]:
table['BANK_ACCOUNT_NUMBER'] = table['BANK_ACCOUNT_NUMBER'].astype('int64')
table.dtypes
mergush = table.merge(df[['BANK_ACCOUNT_NUMBER', 'NEW_LIMIT']], on=['BANK_ACCOUNT_NUMBER'], how='left')
mergush